In [30]:
# reporting/quarterly_report/modules/granting.py
from __future__ import annotations

import logging, sqlite3, datetime
from pathlib import Path
from typing import List
import calendar
import pandas as pd
from datetime import date
from typing import List, Tuple,Union
import numpy as np
from great_tables import GT, loc, style, html
import re
import logging

from ingestion.db_utils import (
    fetch_latest_table_data,
    insert_variable,
    load_report_params,
    fetch_vars_for_report
)


# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('amendments_report.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("Payments")



# our project
from ingestion.db_utils import (
    init_db,                                 # create tables if missing
    fetch_latest_table_data,                 # new version!
    get_alias_last_load,
    get_variable_status, 
    load_report_params                   # to inspect results
)

from reporting.quarterly_report.utils import RenderContext, BaseModule
from reporting.quarterly_report.report_utils.granting_utils import enrich_grants, _ensure_timedelta_cols, _coerce_date_columns
from reporting.quarterly_report.utils import Database, RenderContext


import altair as alt
from altair_saver import save
import selenium.webdriver



# ─────────────────────────────────────────────────────────────
# 2) open DB – change path if you work on a copy
# ─────────────────────────────────────────────────────────────
db_path = "database/reporting.db"
DB_PATH = Path("database/reporting.db")

init_db(db_path=DB_PATH)            # no-op if tables already exist

db = Database(str(DB_PATH))         # thin sqlite3 wrapper
conn = db.conn
report = 'Quarterly_Report'

CALLS_TYPES_LIST = ['STG', 'ADG', 'POC', 'COG', 'SYG', 'StG', 'CoG', 'AdG', 'SyG', 'PoC', 'CSA']
# ──────────────────────────────────────────────────────────────
# HELPERS
# ──────────────────────────────────────────────────────────────

def determine_epoch_year(cutoff_date: pd.Timestamp) -> int:
    """
    Returns the correct reporting year.
    If the cutoff is in January, then we are reporting for the *previous* year.
    """
    return cutoff_date.year - 1 if cutoff_date.month == 1 else cutoff_date.year



def get_scope_start_end(cutoff: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """
    Unified scope logic with year transition:
    • If cutoff is in January → report full previous year
    • Otherwise → return start of year to quarter-end
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        return pd.Timestamp(year=year, month=1, day=1), pd.Timestamp(year=year, month=12, day=31)

    def quarter_end(cutoff: pd.Timestamp) -> pd.Timestamp:
        first_day = cutoff.replace(day=1)
        last_month = first_day - pd.offsets.MonthBegin()
        m = last_month.month

        if m <= 3:
            return pd.Timestamp(year=cutoff.year, month=3, day=31)
        elif m <= 6:
            return pd.Timestamp(year=cutoff.year, month=6, day=30)
        elif m <= 9:
            return pd.Timestamp(year=cutoff.year, month=9, day=30)
        else:
            return pd.Timestamp(year=cutoff.year, month=12, day=31)

    return pd.Timestamp(year=cutoff.year, month=1, day=1), quarter_end(cutoff)



def months_in_scope(cutoff: pd.Timestamp) -> list[str]:
    """
    Returns list of month names from January to last *full* month before cutoff.
    Handles year rollover if cutoff is in January.
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        end_month = 12
    else:
        year = cutoff.year
        end_month = cutoff.month - 1

    months = pd.date_range(
        start=pd.Timestamp(year=year, month=1, day=1),
        end=pd.Timestamp(year=year, month=end_month, day=1),
        freq="MS"
    ).strftime("%B").tolist()

    return months

def determine_po_category(row):

    instrument = str(row.get('Instrument', '')).strip()
    topic = str(row.get('Topic', '')).strip()

    try:
        if topic and any(call_type in topic for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in topic).upper()
            return category
        elif instrument and any(call_type in instrument for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in instrument).upper()
            return category
        return ''
    except Exception as e:
        raise

def determine_po_category_po_list(row):

    summa = str(row.get('PO Purchase Order Item Desc', '')).strip()
    abac = str(row.get('PO ABAC SAP Reference', '')).strip()

    try:
        if summa and any(call_type in summa for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in summa).upper()
            return category
        elif abac and any(call_type in abac for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in abac).upper()
            return category
        return ''
    except Exception as e:
        raise

def extract_project_number(row):
    """
    Extract project number from 'Inv Text' if 'v_check_payment_type' contains RP patterns,
    otherwise return original 'v_check_payment_type' value
    """
    payment_type = row['v_check_payment_type']
    inv_text = row['Inv Text']
    
    # Handle NaN values
    if pd.isna(payment_type):
        return payment_type
    
    # Convert to string to handle any data type
    payment_type_str = str(payment_type)
    
    # Check if the payment_type contains RP patterns:
    # - Original pattern: RP + number + = + FP/IP (e.g., RP4=FP, RP2=IP)
    # - New pattern: RP + number + - + FP/IP (e.g., RP4-FP, RP2-IP)
    rp_patterns = [
        r'RP\d+=(?:FP|IP)',  # Original pattern: RP4=FP, RP2=IP, etc.
        r'RP\d+-(?:FP|IP)'   # New pattern: RP4-FP, RP2-IP, etc.
    ]
    
    # Check if any of the RP patterns match
    has_rp_pattern = any(re.search(pattern, payment_type_str) for pattern in rp_patterns)
    
    if has_rp_pattern:
        # Extract the numerical part from Inv Text column
        if pd.notna(inv_text):
            inv_text_str = str(inv_text).strip()
            # Extract leading digits from Inv Text
            number_match = re.match(r'^(\d+)', inv_text_str)
            if number_match:
                return number_match.group(1)
        
        # If no number found in Inv Text, return original payment_type
        return payment_type
    
    # Return original v_check_payment_type if no RP pattern found
    return payment_type


def map_project_to_call_type(project_num, mapping_dict):
    # If it's a numeric string, try to convert and lookup
    try:
        # Try to convert to int for lookup
        numeric_key = int(project_num)
        if numeric_key in mapping_dict:
            return mapping_dict[numeric_key]
    except (ValueError, TypeError):
        # If conversion fails, it's a non-numeric string like 'EXPERTS'
        pass
    
    # Return original value if no match found
    return project_num

def map_call_type_with_experts(row, grant_map):
    """
    Map call_type based on project_number and Inv Parking Person Id
    """
    project_num = row['project_number']
    contract_type = row['v_payment_type']
    
    # First, try to map using grant_map (convert project_num to int if possible)
    try:
        numeric_key = int(project_num)
        if numeric_key in grant_map:
            return grant_map[numeric_key]
    except (ValueError, TypeError):
        pass
    
    # If project_number is 'EXPERTS', keep it as 'EXPERTS'
    if str(project_num).upper() == 'EXPERTS' or str(contract_type).upper() == 'EXPERTS':
        return 'EXPERTS'
    
    # Return original project_number if no conditions are met
    return project_num

def map_payment_type(row):
    if row['v_payment_type'] == 'Other' and row['Pay Workflow Last AOS Person Id'] == 'WALASOU':
        return 'EXPERTS'
    return row['v_payment_type']

# Instead, handle conversion in the mapping function
def safe_map_project_to_call_type(project_num, mapping_dict):
    """
    Maps project number to call type, handles all data type issues internally
    """
    try:
        # Handle NaN values
        if pd.isna(project_num):
            return None
            
        # Convert whatever format to integer for lookup
        if isinstance(project_num, str):
            # Handle strings like '4500053782.0'
            if project_num.endswith('.0'):
                numeric_key = int(project_num[:-2])
            else:
                numeric_key = int(float(project_num))
        else:
            # Handle numeric values (float/int)
            numeric_key = int(float(project_num))
            
        # Lookup in mapping dictionary
        if numeric_key in mapping_dict:
            result = mapping_dict[numeric_key]
            if pd.notna(result) and result != '':
                return result
                
    except (ValueError, TypeError, OverflowError):
        # Any conversion error, return None
        pass
    
    return None

# Apply mapping without converting the whole column
def apply_conditional_mapping(row):
    current_call_type = row['call_type']
    po_key = row['PO Purchase Order Key']  # Use as-is, no conversion
    
    should_map = (
        pd.isna(current_call_type) or 
        current_call_type == '' or 
        current_call_type not in CALLS_TYPES_LIST or 
        current_call_type in ['EXPERTS', 'CSA']
    )
    
    if should_map:
        mapped_value = safe_map_project_to_call_type(po_key, po_map)
        return mapped_value if mapped_value is not None else current_call_type
    else:
        return current_call_type



In [37]:
PAYMENTS_ALIAS = "payments_summa"
CALLS_ALIAS = 'call_overview'
PAYMENTS_TIMES_ALIAS = 'payments_summa_time'
PO_ALIAS = 'c0_po_summa'
FORECAST_ALIAS = 'forecast'

cutoff = pd.to_datetime("2025-06-01")
report_params = load_report_params(report_name=report, db_path=db_path)


table_colors = report_params.get('TABLE_COLORS', {})
BLUE = table_colors.get("BLUE", "#004A99")
LIGHT_BLUE = table_colors.get("LIGHT_BLUE", "#d6e6f4")
DARK_BLUE = table_colors.get("DARK_BLUE", "#01244B")
SUB_TOTAL_BACKGROUND = table_colors.get("subtotal_background_color", "#E6E6FA")

df_paym = fetch_latest_table_data(conn, PAYMENTS_ALIAS, cutoff)
df_paym_times = fetch_latest_table_data(conn, PAYMENTS_TIMES_ALIAS, cutoff)
df_calls =  fetch_latest_table_data(conn, CALLS_ALIAS , cutoff)
df_po = fetch_latest_table_data(conn, PO_ALIAS, cutoff)
df_forecast = fetch_latest_table_data(conn, FORECAST_ALIAS, cutoff)

DEBUG:root:Fetching latest data for table_alias: payments_summa, cutoff: 2025-06-01T00:00:00
DEBUG:root:Upload log query results for payments_summa: [('2025-06-04T09:38:45.291982', 16)]
DEBUG:root:Checking upload_id: 16, uploaded_at: 2025-06-04T09:38:45.291982
DEBUG:root:Fetched 6391 rows from payments_summa with upload_id 16
DEBUG:root:Fetching latest data for table_alias: payments_summa_time, cutoff: 2025-06-01T00:00:00
DEBUG:root:Upload log query results for payments_summa_time: [('2025-06-04T09:39:00.815408', 17)]
DEBUG:root:Checking upload_id: 17, uploaded_at: 2025-06-04T09:39:00.815408
DEBUG:root:Fetched 4992 rows from payments_summa_time with upload_id 17
DEBUG:root:Fetching latest data for table_alias: call_overview, cutoff: 2025-06-01T00:00:00
DEBUG:root:Upload log query results for call_overview: [('2025-06-04T09:43:14.796908', 18)]
DEBUG:root:Checking upload_id: 18, uploaded_at: 2025-06-04T09:43:14.796908
DEBUG:root:Fetched 13295 rows from call_overview with upload_id 18
DEB

In [38]:
df_paym['v_payment_type'] = df_paym.apply(map_payment_type, axis=1)
# Filter the dataframe
df_paym = df_paym[df_paym['Pay Document Type Desc'].isin(['Payment Directive', 'Exp Pre-financing'])]
# Keep all rows where v_payment_type is not 'Other'
df_paym = df_paym[df_paym['v_payment_type'] != 'Other']
df_paym = df_paym[df_paym['Pay Payment Key'].notnull()]

df_paym['project_number'] = df_paym.apply(extract_project_number, axis=1)

# Assuming your DataFrame is called 'df'
df_calls['CALL_TYPE'] = df_calls.apply(determine_po_category, axis=1)
grant_map = df_calls.set_index('Grant Number')['CALL_TYPE'].to_dict()

#PO ORDERS MAP
df_po['CALL_TYPE']  = df_po.apply(determine_po_category_po_list, axis=1)

po_map = df_po[
    df_po['CALL_TYPE'].notna() & 
    (df_po['CALL_TYPE'].str.strip() != '')
].set_index('PO Purchase Order Key')['CALL_TYPE'].to_dict()

# Apply the mapping
df_paym['call_type'] = df_paym['project_number'].apply(lambda x: map_project_to_call_type(x, grant_map))
df_paym['call_type'] = df_paym.apply(lambda row: map_call_type_with_experts(row, grant_map), axis=1)


# Clean call_type column only (not PO keys)
df_paym['call_type'] = df_paym['call_type'].astype(str).str.strip().replace(['nan', ''], np.nan)
# Apply the mapping
df_paym['call_type'] = df_paym.apply(apply_conditional_mapping, axis=1)
# This preserves NaN values as NaN instead of causing errors
df_paym['PO Purchase Order Key'] = pd.to_numeric(df_paym['PO Purchase Order Key'], errors='coerce').astype('Int64')

df_paym['Pay Document Date (dd/mm/yyyy)'] = pd.to_datetime(
    df_paym['Pay Document Date (dd/mm/yyyy)'], 
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

quarter_dates = get_scope_start_end(cutoff=cutoff)
last_valid_date = quarter_dates[1]

df_paym = df_paym[
    df_paym['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
].copy()

df_paym = df_paym[df_paym['call_type'] != 'CSA']

In [39]:
# Step 1: Create v_payment_in_time column in df_paym_times
print("=== STEP 1: Creating v_payment_in_time column ===")

# Convert Pay Delay Late Payment Flag (Y/N) to 1/0
df_paym_times['v_payment_in_time'] = df_paym_times['Pay Delay Late Payment Flag (Y/N)'].apply(
    lambda x: 1 if x == 'N' else 0
)

print("v_payment_in_time value counts:")
print(df_paym_times['v_payment_in_time'].value_counts())
print("\nOriginal flag vs new column:")
print(df_paym_times[['Pay Delay Late Payment Flag (Y/N)', 'v_payment_in_time']].value_counts())

# Step 2: Clean Pay Payment Key and create mappings
print("\n=== STEP 2: Creating payment mappings ===")

# Filter df_paym_times to only include rows we need for mapping
df_times_clean = df_paym_times.dropna(subset=['Pay Payment Key']).copy()

# Clean Pay Payment Key for mapping (convert to integers)
def safe_convert_to_int(value):
    """Safely convert payment key to integer"""
    try:
        if pd.isna(value):
            return None
        if isinstance(value, str):
            # Handle strings like '2551003294.0'
            if value.endswith('.0'):
                return int(value[:-2])
            else:
                return int(float(value))
        else:
            return int(float(value))
    except (ValueError, TypeError, OverflowError):
        return None

# Convert Pay Payment Key to integers for mapping (keep all rows)
df_times_clean['Pay_Payment_Key_Int'] = df_times_clean['Pay Payment Key'].apply(safe_convert_to_int)

# Count conversion issues but don't drop rows
conversion_failed = df_times_clean['Pay_Payment_Key_Int'].isna().sum()
conversion_success = df_times_clean['Pay_Payment_Key_Int'].notna().sum()

print(f"Payment key conversions - Success: {conversion_success}, Failed: {conversion_failed}")

# Create mappings only from successfully converted keys (but keep all rows in dataframe)
valid_conversions = df_times_clean['Pay_Payment_Key_Int'].notna()
mapping_data = df_times_clean[valid_conversions].copy()
mapping_data['Pay_Payment_Key_Int'] = mapping_data['Pay_Payment_Key_Int'].astype(int)

print(f"Payment times data: {len(df_times_clean)} total rows, {len(mapping_data)} usable for mapping")

# Create the three mappings (only from rows with valid conversions)
payment_key_to_ttp_gross = mapping_data.set_index('Pay_Payment_Key_Int')['Pay Delay With Suspension'].to_dict()
payment_key_to_ttp_net = mapping_data.set_index('Pay_Payment_Key_Int')['Pay Delay Without Suspension'].to_dict()
payment_key_to_payment_in_time = mapping_data.set_index('Pay_Payment_Key_Int')['v_payment_in_time'].to_dict()

print(f"TTP Gross mapping created: {len(payment_key_to_ttp_gross)} entries")
print(f"TTP Net mapping created: {len(payment_key_to_ttp_net)} entries")
print(f"Payment in time mapping created: {len(payment_key_to_payment_in_time)} entries")

# Step 3: Split dataframe and apply mappings selectively
print("\n=== STEP 3: Split, Map, and Merge Strategy ===")

# Check conditions in df_paym
exp_prefi_mask = df_paym['Pay Document Type Desc'] == 'Exp Pre-financing'
payment_directive_mask = df_paym['Pay Document Type Desc'] == 'Payment Directive'
other_mask = ~(exp_prefi_mask | payment_directive_mask)

print(f"Rows with 'Exp Pre-financing': {exp_prefi_mask.sum()}")
print(f"Rows with 'Payment Directive': {payment_directive_mask.sum()}")
print(f"Other document types: {other_mask.sum()}")
print(f"Total rows in df_paym: {len(df_paym)}")

# Step 3.1: Split the dataframe
df_exp_prefi = df_paym[exp_prefi_mask].copy()
df_payment_directive = df_paym[payment_directive_mask].copy()
df_other = df_paym[other_mask].copy()

print(f"\nDataframes split:")
print(f"- Exp Pre-financing: {len(df_exp_prefi)} rows")
print(f"- Payment Directive: {len(df_payment_directive)} rows")
print(f"- Other types: {len(df_other)} rows")

# Step 3.2: Apply mapping ONLY to Exp Pre-financing dataframe
if len(df_exp_prefi) > 0:
    print("\nApplying mappings to Exp Pre-financing dataframe...")
    
    # Mapping function for payment data
    def map_payment_data(pay_key, mapping_dict):
        """Map payment key to corresponding value"""
        try:
            if pd.isna(pay_key):
                return np.nan
                
            # Convert pay_key to int for lookup
            if isinstance(pay_key, str):
                if pay_key.endswith('.0'):
                    numeric_key = int(pay_key[:-2])
                else:
                    numeric_key = int(float(pay_key))
            else:
                numeric_key = int(float(pay_key))
                
            # Lookup in mapping
            if numeric_key in mapping_dict:
                return mapping_dict[numeric_key]
            else:
                return np.nan
                
        except (ValueError, TypeError, OverflowError):
            return np.nan
    
    # Add the three new columns to Exp Pre-financing dataframe ONLY
    df_exp_prefi['v_TTP_GROSS'] = df_exp_prefi['Pay Payment Key'].apply(
        lambda x: map_payment_data(x, payment_key_to_ttp_gross)
    )
    
    df_exp_prefi['v_TTP_NET'] = df_exp_prefi['Pay Payment Key'].apply(
        lambda x: map_payment_data(x, payment_key_to_ttp_net)
    )
    
    df_exp_prefi['v_payment_in_time'] = df_exp_prefi['Pay Payment Key'].apply(
        lambda x: map_payment_data(x, payment_key_to_payment_in_time)
    )
    
    print("Mapping applied to Exp Pre-financing rows!")
    
    # Show sample of what was mapped
    sample_mapped = df_exp_prefi[['Pay Payment Key', 'Pay Document Type Desc', 'v_TTP_GROSS', 'v_TTP_NET', 'v_payment_in_time']].head()
    print("Sample mapped rows:")
    print(sample_mapped)
else:
    print("No Exp Pre-financing rows to map!")

# Step 3.3: Payment Directive dataframe - ADD columns but DON'T change any existing values
if len(df_payment_directive) > 0:
    print("\nPreserving Payment Directive dataframe completely...")
    
    # Check if these columns already exist in Payment Directive rows
    if 'v_TTP_GROSS' in df_payment_directive.columns:
        print("v_TTP_GROSS already exists in Payment Directive - preserving original values")
    else:
        # Only add columns if they don't exist, with NaN values
        df_payment_directive['v_TTP_GROSS'] = np.nan
        
    if 'v_TTP_NET' in df_payment_directive.columns:
        print("v_TTP_NET already exists in Payment Directive - preserving original values")
    else:
        df_payment_directive['v_TTP_NET'] = np.nan
        
    if 'v_payment_in_time' in df_payment_directive.columns:
        print("v_payment_in_time already exists in Payment Directive - preserving original values")
    else:
        df_payment_directive['v_payment_in_time'] = np.nan
    
    print("Payment Directive rows completely preserved!")

# Step 3.4: Handle other document types
if len(df_other) > 0:
    print("Adding columns to other document types with NaN values...")
    df_other['v_TTP_GROSS'] = np.nan
    df_other['v_TTP_NET'] = np.nan
    df_other['v_payment_in_time'] = np.nan

# Step 3.5: Merge all dataframes back together
print("\nMerging dataframes back together...")

dataframes_to_merge = []
if len(df_exp_prefi) > 0:
    dataframes_to_merge.append(df_exp_prefi)
if len(df_payment_directive) > 0:
    dataframes_to_merge.append(df_payment_directive)
if len(df_other) > 0:
    dataframes_to_merge.append(df_other)

if dataframes_to_merge:
    df_paym = pd.concat(dataframes_to_merge, ignore_index=True)
    print(f"Dataframes merged successfully! Final shape: {df_paym.shape}")
else:
    print("Warning: No dataframes to merge!")

# Step 4: Check results
print("\n=== STEP 4: Results ===")
print("v_TTP_GROSS value counts:")
print(df_paym['v_TTP_GROSS'].value_counts(dropna=False))
print("\nv_TTP_NET value counts:")
print(df_paym['v_TTP_NET'].value_counts(dropna=False))
print("\nv_payment_in_time value counts:")
print(df_paym['v_payment_in_time'].value_counts(dropna=False))

# Show sample of mapped rows
print("\nSample of mapped rows:")
mapped_sample = df_paym[exp_prefi_mask & df_paym['v_TTP_GROSS'].notna()][
    ['Pay Payment Key', 'Pay Document Type Desc', 'v_TTP_GROSS', 'v_TTP_NET', 'v_payment_in_time']
].head()
print(mapped_sample)

=== STEP 1: Creating v_payment_in_time column ===
v_payment_in_time value counts:
v_payment_in_time
1    4776
0     216
Name: count, dtype: int64

Original flag vs new column:
Pay Delay Late Payment Flag (Y/N)  v_payment_in_time
N                                  1                    4776
Y                                  0                     216
Name: count, dtype: int64

=== STEP 2: Creating payment mappings ===
Payment key conversions - Success: 4992, Failed: 0
Payment times data: 4992 total rows, 4992 usable for mapping
TTP Gross mapping created: 4992 entries
TTP Net mapping created: 4992 entries
Payment in time mapping created: 4992 entries

=== STEP 3: Split, Map, and Merge Strategy ===
Rows with 'Exp Pre-financing': 581
Rows with 'Payment Directive': 4309
Other document types: 0
Total rows in df_paym: 4890

Dataframes split:
- Exp Pre-financing: 581 rows
- Payment Directive: 4309 rows
- Other types: 0 rows

Applying mappings to Exp Pre-financing dataframe...
Mapping applied to

C:\Users\vinci\AppData\Local\Temp\ipykernel_32600\632175654.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mapped_sample = df_paym[exp_prefi_mask & df_paym['v_TTP_GROSS'].notna()][


In [22]:
df_paym.to_excel('paym.xlsx')

In [40]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Tuple

# Existing periodicity functions
def determine_epoch_year(cutoff_date: pd.Timestamp) -> int:
    """
    Returns the correct reporting year.
    If the cutoff is in January, then we are reporting for the *previous* year.
    """
    return cutoff_date.year - 1 if cutoff_date.month == 1 else cutoff_date.year

def get_scope_start_end(cutoff: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """
    Unified scope logic with year transition:
    • If cutoff is in January → report full previous year
    • Otherwise → return start of year to quarter-end
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        return pd.Timestamp(year=year, month=1, day=1), pd.Timestamp(year=year, month=12, day=31)

    def quarter_end(cutoff: pd.Timestamp) -> pd.Timestamp:
        first_day = cutoff.replace(day=1)
        last_month = first_day - pd.offsets.MonthBegin()
        m = last_month.month

        if m <= 3:
            return pd.Timestamp(year=cutoff.year, month=3, day=31)
        elif m <= 6:
            return pd.Timestamp(year=cutoff.year, month=6, day=30)
        elif m <= 9:
            return pd.Timestamp(year=cutoff.year, month=9, day=30)
        else:
            return pd.Timestamp(year=cutoff.year, month=12, day=31)

    return pd.Timestamp(year=cutoff.year, month=1, day=1), quarter_end(cutoff)

def months_in_scope(cutoff: pd.Timestamp) -> list[str]:
    """
    Returns list of month names from January to last *full* month before cutoff.
    Handles year rollover if cutoff is in January.
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        end_month = 12
    else:
        year = cutoff.year
        end_month = cutoff.month - 1

    months = pd.date_range(
        start=pd.Timestamp(year=year, month=1, day=1),
        end=pd.Timestamp(year=year, month=end_month, day=1),
        freq="MS"
    ).strftime("%B").tolist()

    return months

def create_quarterly_payment_tables(df_paym, cutoff_date=None):
    """
    Create quarterly payment tables matching the format from the Excel file
    - Amount summing: All v_amount_to_sum per payment key, regrouped by fund source
    - Number of payments: Count unique Pay Payment Key occurrences (deduplicated)
    - Assumes df_paym is already filtered for the correct time scope
    """
    
    print("=== QUARTERLY PAYMENT TABLES GENERATION ===")
    
    # Step 1: Set cutoff date for metadata
    if cutoff_date is None:
        cutoff_date = pd.Timestamp.now()
    elif isinstance(cutoff_date, str):
        cutoff_date = pd.Timestamp(cutoff_date)
    
    print(f"Cutoff date: {cutoff_date}")
    
    # Step 2: Get reporting metadata (for reference)
    reporting_year = determine_epoch_year(cutoff_date)
    scope_start, scope_end = get_scope_start_end(cutoff_date)
    months_in_report = months_in_scope(cutoff_date)
    
    print(f"Reporting year: {reporting_year}")
    print(f"Expected scope: {scope_start} to {scope_end}")
    print(f"Note: Assuming df_paym is already filtered for this scope")
    
    # Step 3: Validate required columns
    required_columns = [
        'Pay Payment Key', 
        'v_amount_to_sum', 
        'Fund Source',
        'v_payment_type', 
        'Pay Document Date (dd/mm/yyyy)',
        'Programme'
    ]
    
    # Check for optional call_type column
    optional_columns = ['call_type', 'Call Type', 'v_call_type']
    call_type_col = None
    for col in optional_columns:
        if col in df_paym.columns:
            call_type_col = col
            print(f"Found call type column: {col}")
            break
    
    if call_type_col:
        required_columns.append(call_type_col)
    else:
        print("No call_type column found - will use Fund Source only")
    
    missing_columns = [col for col in required_columns if col not in df_paym.columns]
    if missing_columns:
        print(f"ERROR: Missing required columns: {missing_columns}")
        return None
    
    print("✓ All required columns present")
    
    # Step 4: Create working dataframe (skip date filtering since already done)
    df_work = df_paym[required_columns].copy()
    
    # Check for any remaining invalid dates
    invalid_dates = df_work['Pay Document Date (dd/mm/yyyy)'].isna().sum()
    if invalid_dates > 0:
        print(f"WARNING: {invalid_dates} rows with invalid dates found, removing them")
        df_work = df_work.dropna(subset=['Pay Document Date (dd/mm/yyyy)'])
    
    print(f"Working dataset: {len(df_work)} rows")
    
    if len(df_work) == 0:
        print("ERROR: No data available after validation")
        return None
    
    # Create quarter and year columns
    df_work['Quarter'] = df_work['Pay Document Date (dd/mm/yyyy)'].dt.to_period('Q')
    df_work['Year'] = df_work['Pay Document Date (dd/mm/yyyy)'].dt.year
    df_work['Quarter_Label'] = df_work['Quarter'].astype(str)
    
    print(f"Actual date range: {df_work['Pay Document Date (dd/mm/yyyy)'].min()} to {df_work['Pay Document Date (dd/mm/yyyy)'].max()}")
    print(f"Quarters found: {sorted(df_work['Quarter_Label'].unique())}")
    
    # Step 5: Map payment types and fund sources
    payment_type_mapping = {
        'IP': 'Interim Payments',
        'FP': 'Final Payments', 
        'PF': 'Pre-financing',
        'EXPERTS': 'Experts and Support'
    }
    
    # Keep original fund sources for now (don't map to C1/E0 yet)
    df_work['Payment_Type_Desc'] = df_work['v_payment_type'].map(payment_type_mapping)
    
    # Use call_type if available, otherwise use Fund Source
    if call_type_col:
        df_work['Call_Type_Display'] = df_work[call_type_col]
        print(f"Call types found: {sorted(df_work['Call_Type_Display'].unique())}")
    else:
        df_work['Call_Type_Display'] = df_work['Fund Source']
        print(f"Using Fund Source as call type: {sorted(df_work['Call_Type_Display'].unique())}")
    
    # Handle unmapped payment types
    unmapped_payments = df_work[df_work['Payment_Type_Desc'].isna()]['v_payment_type'].unique()
    if len(unmapped_payments) > 0:
        print(f"WARNING: Unmapped payment types found: {unmapped_payments}")
        # Keep unmapped ones with their original value
        df_work['Payment_Type_Desc'] = df_work['Payment_Type_Desc'].fillna(df_work['v_payment_type'])
    
    # Step 6: Split by Programme (H2020 and HEU)
    programmes = df_work['Programme'].unique()
    print(f"Programmes found: {programmes}")
    
    results = {
        'metadata': {
            'cutoff_date': cutoff_date,
            'reporting_year': reporting_year,
            'scope_start': scope_start,
            'scope_end': scope_end,
            'months_in_scope': months_in_report,
            'actual_date_range': {
                'start': df_work['Pay Document Date (dd/mm/yyyy)'].min(),
                'end': df_work['Pay Document Date (dd/mm/yyyy)'].max()
            },
            'call_type_column': call_type_col,
            'has_call_types': call_type_col is not None
        },
        'tables': {}
    }
    
    for programme in programmes:
        if programme not in ['H2020', 'HEU']:
            print(f"Skipping programme: {programme}")
            continue
            
        print(f"\n=== Processing {programme} ===")
        df_prog = df_work[df_work['Programme'] == programme].copy()
        
        if len(df_prog) == 0:
            print(f"No data for {programme}")
            continue
        
        # Create aggregation tables
        tables = create_programme_tables(df_prog, programme, reporting_year)
        results['tables'][programme] = tables
    
    return results

def create_programme_tables(df_prog, programme_name, reporting_year):
    """
    Create all payment type tables for a specific programme
    """
    
    tables = {}
    
    # Get unique payment types in this programme
    payment_types = df_prog['Payment_Type_Desc'].dropna().unique()
    
    for payment_type in payment_types:
        print(f"  Creating table for: {payment_type}")
        
        df_type = df_prog[df_prog['Payment_Type_Desc'] == payment_type].copy()
        
        if len(df_type) == 0:
            continue
            
        # Create quarterly aggregation
        quarterly_table = create_quarterly_aggregation(df_type, payment_type, reporting_year)
        tables[payment_type] = quarterly_table
    
    # Create overall summary table
    print(f"  Creating overall summary table")
    overall_table = create_quarterly_aggregation(df_prog, "All Payments", reporting_year)
    tables['All_Payments'] = overall_table
    
    return tables

def create_quarterly_aggregation(df_type, payment_type_name, reporting_year):
    """
    Create quarterly aggregation table for a specific payment type
    - Amounts: Sum all v_amount_to_sum (including by call type/fund source)
    - Transactions: Count unique Pay Payment Key
    - VOBU/EFTA: Sum only EFTA and VOBU fund sources
    """
    
    # Create base aggregation structure
    agg_data = []
    
    # Get all quarters in the data
    quarters = sorted(df_type['Quarter'].unique())
    
    for quarter in quarters:
        df_q = df_type[df_type['Quarter'] == quarter].copy()
        
        # Get call types for this quarter (using Call_Type_Display)
        call_types = df_q['Call_Type_Display'].unique()
        
        quarter_row = {
            'Quarter': str(quarter),
            'Quarter_Short': f"{quarter.quarter}Q{quarter.year}",
            'Year': quarter.year,
            'Payment_Type': payment_type_name,
            'Reporting_Year': reporting_year
        }
        
        # AMOUNTS: Sum all v_amount_to_sum by call type
        total_amount_all_types = 0
        vobu_efta_amount_all_types = 0
        
        for call_type in call_types:
            df_call_type = df_q[df_q['Call_Type_Display'] == call_type]
            
            # Total amount for this call type
            total_amount = df_call_type['v_amount_to_sum'].sum()
            quarter_row[f'Total_Amount_{call_type}'] = total_amount
            total_amount_all_types += total_amount
            
            # VOBU/EFTA amount: Only sum EFTA and VOBU fund sources
            df_vobu_efta = df_call_type[df_call_type['Fund Source'].isin(['VOBU', 'EFTA'])]
            vobu_efta_amount = df_vobu_efta['v_amount_to_sum'].sum()
            quarter_row[f'VOBU_EFTA_Amount_{call_type}'] = vobu_efta_amount
            vobu_efta_amount_all_types += vobu_efta_amount
            
            # TRANSACTIONS: Count unique Pay Payment Key for this call type
            unique_transactions_call_type = df_call_type['Pay Payment Key'].nunique()
            quarter_row[f'No_of_Transactions_{call_type}'] = unique_transactions_call_type
        
        # TRANSACTIONS: Count unique Pay Payment Key (deduplicated across all call types)
        unique_transactions = df_q['Pay Payment Key'].nunique()
        quarter_row['No_of_Transactions'] = unique_transactions
        
        # OVERALL TOTALS
        quarter_row['Total_Amount'] = total_amount_all_types
        quarter_row['VOBU_EFTA_Amount'] = vobu_efta_amount_all_types
        
        agg_data.append(quarter_row)
    
    # Convert to DataFrame
    df_result = pd.DataFrame(agg_data)
    
    # Add total row
    if len(df_result) > 0:
        total_row = create_total_row(df_type, df_result, payment_type_name, reporting_year)
        df_result = pd.concat([df_result, total_row], ignore_index=True)
    
    return df_result

def create_total_row(df_type, df_result, payment_type_name, reporting_year):
    """
    Create total row for the aggregation table with VOBU/EFTA logic and transaction counts by call type
    """
    
    total_row = {
        'Quarter': 'Total',
        'Quarter_Short': 'Total',
        'Year': reporting_year,
        'Payment_Type': payment_type_name,
        'Reporting_Year': reporting_year
    }
    
    # Sum all amount columns (exclude Total row if it exists)
    df_data_only = df_result[df_result['Quarter'] != 'Total']
    
    # Sum individual call type amounts
    amount_cols = [col for col in df_result.columns if 'Amount' in col and col not in ['Total_Amount', 'VOBU_EFTA_Amount']]
    for col in amount_cols:
        total_row[col] = df_data_only[col].sum()
    
    # Calculate VOBU/EFTA total from original data (not summing quarterly totals to avoid double counting)
    df_vobu_efta = df_type[df_type['Fund Source'].isin(['VOBU', 'EFTA'])]
    total_row['VOBU_EFTA_Amount'] = df_vobu_efta['v_amount_to_sum'].sum()
    
    # Calculate transaction counts by call type from original data
    call_types = df_type['Call_Type_Display'].unique()
    for call_type in call_types:
        df_call_type = df_type[df_type['Call_Type_Display'] == call_type]
        total_row[f'No_of_Transactions_{call_type}'] = df_call_type['Pay Payment Key'].nunique()
    
    # Overall total amount
    total_row['Total_Amount'] = df_type['v_amount_to_sum'].sum()
    
    # Sum unique transactions across all quarters (deduplicated at total level)
    total_row['No_of_Transactions'] = df_type['Pay Payment Key'].nunique()
    
    return pd.DataFrame([total_row])

def format_table_for_great_tables(df_table, payment_type, programme, repeat_quarter=True):
    """
    Format table for great_tables library - creates clean pandas DataFrame
    Structure exactly like Excel: Quarter | Metric | ADG | COG | POC | STG | SYG | Total
    
    Args:
        repeat_quarter (bool): If True, repeat quarter value in each row. If False, show only once per group.
                              True is recommended for great_tables compatibility.
    """
    
    if len(df_table) == 0:
        return pd.DataFrame()
    
    # Separate data rows from total row
    df_data = df_table[df_table['Quarter'] != 'Total'].copy()
    df_total = df_table[df_table['Quarter'] == 'Total'].copy()
    
    if len(df_data) == 0:
        return pd.DataFrame()
    
    # Get unique quarters and call types from the data
    quarters = sorted(df_data['Quarter_Short'].unique())
    
    # Extract call type columns from the dataframe
    call_type_cols = [col for col in df_data.columns if col.startswith('Total_Amount_') and not col.endswith('Amount')]
    call_types = sorted([col.replace('Total_Amount_', '') for col in call_type_cols if col != 'Total_Amount'])
    
    print(f"  Formatting for great_tables - Call types: {call_types}, Quarters: {quarters}")
    print(f"  Quarter repeat mode: {repeat_quarter}")
    
    # Create the structure for great_tables - Quarter and Metric as separate columns
    table_data = []
    
    # === PROCESS EACH QUARTER ===
    for quarter in quarters:
        quarter_data = df_data[df_data['Quarter_Short'] == quarter]
        
        if len(quarter_data) == 0:
            continue
            
        # ROW 1: Total Amount for this quarter
        total_amount_row = {
            'Quarter': quarter, 
            'Metric': 'Total Amount'
        }
        
        for call_type in call_types:
            amount_col = f'Total_Amount_{call_type}'
            total_amount_row[call_type] = quarter_data[amount_col].iloc[0] if amount_col in quarter_data.columns else 0
        
        total_amount_row['Total'] = quarter_data['Total_Amount'].iloc[0]
        table_data.append(total_amount_row)
        
        # ROW 2: Out of Which VOBU/EFTA for this quarter
        vobu_efta_row = {
            'Quarter': quarter if repeat_quarter else '', 
            'Metric': 'Out of Which VOBU/EFTA'
        }
        
        for call_type in call_types:
            vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
            vobu_efta_row[call_type] = quarter_data[vobu_efta_col].iloc[0] if vobu_efta_col in quarter_data.columns else 0
        
        vobu_efta_row['Total'] = quarter_data['VOBU_EFTA_Amount'].iloc[0]
        table_data.append(vobu_efta_row)
        
        # ROW 3: No of Transactions for this quarter
        transactions_row = {
            'Quarter': quarter if repeat_quarter else '', 
            'Metric': 'No of Transactions'
        }
        
        for call_type in call_types:
            transactions_col = f'No_of_Transactions_{call_type}'
            transactions_row[call_type] = quarter_data[transactions_col].iloc[0] if transactions_col in quarter_data.columns else 0
        
        transactions_row['Total'] = quarter_data['No_of_Transactions'].iloc[0]
        table_data.append(transactions_row)
    
    # === TOTAL ROWS (from df_total) ===
    if len(df_total) > 0:
        
        # TOTAL ROW 1: Total Amount
        total_amount_row = {
            'Quarter': 'Total', 
            'Metric': 'Total Amount'
        }
        
        for call_type in call_types:
            amount_col = f'Total_Amount_{call_type}'
            total_amount_row[call_type] = df_total[amount_col].iloc[0] if amount_col in df_total.columns else 0
        
        total_amount_row['Total'] = df_total['Total_Amount'].iloc[0]
        table_data.append(total_amount_row)
        
        # TOTAL ROW 2: Out of Which VOBU/EFTA
        total_vobu_efta_row = {
            'Quarter': 'Total' if repeat_quarter else '', 
            'Metric': 'Out of Which VOBU/EFTA'
        }
        
        for call_type in call_types:
            vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
            total_vobu_efta_row[call_type] = df_total[vobu_efta_col].iloc[0] if vobu_efta_col in df_total.columns else 0
        
        total_vobu_efta_row['Total'] = df_total['VOBU_EFTA_Amount'].iloc[0]
        table_data.append(total_vobu_efta_row)
        
        # TOTAL ROW 3: No of Transactions
        total_transactions_row = {
            'Quarter': 'Total' if repeat_quarter else '', 
            'Metric': 'No of Transactions'
        }
        
        for call_type in call_types:
            transactions_col = f'No_of_Transactions_{call_type}'
            total_transactions_row[call_type] = df_total[transactions_col].iloc[0] if transactions_col in df_total.columns else 0
        
        total_transactions_row['Total'] = df_total['No_of_Transactions'].iloc[0]
        table_data.append(total_transactions_row)
    
    # Convert to DataFrame
    great_tables_df = pd.DataFrame(table_data)
    
    # Reorder columns: Quarter, Metric, then call types in alphabetical order, then Total
    column_order = ['Quarter', 'Metric'] + call_types + ['Total']
    great_tables_df = great_tables_df[column_order]
    
    return great_tables_df

# Main execution function with periodicity integration
def generate_all_quarterly_tables(df_paym, cutoff_date=None):
    """
    Main function to generate all quarterly payment tables with proper periodicity handling
    """
    
    print("Starting quarterly table generation with periodicity logic...")
    
    if cutoff_date is not None:
        print(f"Using provided cutoff date: {cutoff_date}")
    else:
        cutoff_date = pd.Timestamp.now()
        print(f"Using current date as cutoff: {cutoff_date}")
    
    # Generate tables with scope filtering
    results = create_quarterly_payment_tables(df_paym, cutoff_date)
    
    if results is None:
        return None
    
    # Format for display
    formatted_results = format_quarterly_tables_for_display(results)
    
    # Display summary
    print("\n=== GENERATION COMPLETE ===")
    print(f"Reporting for: {results['metadata']['reporting_year']}")
    print(f"Scope: {results['metadata']['scope_start']} to {results['metadata']['scope_end']}")
    
    if 'tables' in results:
        for programme, tables in results['tables'].items():
            print(f"\n{programme} Programme:")
            for payment_type, table in tables.items():
                data_rows = len(table[table['Quarter'] != 'Total']) if len(table) > 0 else 0
                print(f"  - {payment_type}: {data_rows} quarters")
    
    return formatted_results

def format_quarterly_tables_for_great_tables(results):
    """
    Format the results for great_tables library - clean pandas DataFrames
    """
    
    if 'tables' not in results:
        return results
        
    formatted_results = {
        'metadata': results['metadata'],
        'great_tables': {}
    }
    
    for programme, tables in results['tables'].items():
        formatted_results['great_tables'][programme] = {}
        
        for payment_type, df_table in tables.items():
            # Create great_tables format
            gt_table = format_table_for_great_tables(df_table, payment_type, programme)
            formatted_results['great_tables'][programme][payment_type] = gt_table
    
    return formatted_results

# Main execution function with great_tables output
def generate_all_quarterly_tables(df_paym, cutoff_date=None):
    """
    Main function to generate all quarterly payment tables for great_tables
    """
    
    print("Starting quarterly table generation for great_tables...")
    
    if cutoff_date is not None:
        print(f"Using provided cutoff date: {cutoff_date}")
    else:
        cutoff_date = pd.Timestamp.now()
        print(f"Using current date as cutoff: {cutoff_date}")
    
    # Generate tables with scope filtering
    results = create_quarterly_payment_tables(df_paym, cutoff_date)
    
    if results is None:
        return None
    
    # Format for great_tables
    formatted_results = format_quarterly_tables_for_great_tables(results)
    
    # Display summary
    print("\n=== GENERATION COMPLETE ===")
    print(f"Reporting for: {results['metadata']['reporting_year']}")
    print(f"Scope: {results['metadata']['scope_start']} to {results['metadata']['scope_end']}")
    print(f"VOBU/EFTA aggregation: Only EFTA and VOBU fund sources included")
    
    if 'tables' in results:
        for programme, tables in results['tables'].items():
            print(f"\n{programme} Programme:")
            for payment_type, table in tables.items():
                data_rows = len(table[table['Quarter'] != 'Total']) if len(table) > 0 else 0
                print(f"  - {payment_type}: {data_rows} quarters")
    
    return formatted_results

# Updated utility functions for great_tables
def get_great_table(formatted_results, programme, payment_type):
    """
    Get a specific table formatted for great_tables
    """
    try:
        return formatted_results['great_tables'][programme][payment_type]
    except KeyError:
        print(f"Table not found: {programme} - {payment_type}")
        available_programmes = list(formatted_results.get('great_tables', {}).keys())
        print(f"Available programmes: {available_programmes}")
        if programme in formatted_results.get('great_tables', {}):
            available_payment_types = list(formatted_results['great_tables'][programme].keys())
            print(f"Available payment types for {programme}: {available_payment_types}")
        return pd.DataFrame()

def get_summary_table(formatted_results, programme):
    """
    Get the summary table that includes all payment types (including experts)
    """
    return get_great_table(formatted_results, programme, 'All_Payments')

def create_comprehensive_summary_table(results, programme):
    """
    Create a comprehensive summary table showing all payment types in one view
    """
    
    if 'tables' not in results or programme not in results['tables']:
        print(f"No data found for programme: {programme}")
        return pd.DataFrame()
    
    programme_tables = results['tables'][programme]
    
    # Initialize summary data
    summary_data = []
    
    # Get all call types from any table
    all_call_types = set()
    for payment_type, table in programme_tables.items():
        if payment_type != 'All_Payments' and len(table) > 0:
            total_row = table[table['Quarter'] == 'Total']
            if len(total_row) > 0:
                call_type_cols = [col for col in total_row.columns if col.startswith('Total_Amount_')]
                call_types = [col.replace('Total_Amount_', '') for col in call_type_cols]
                all_call_types.update(call_types)
    
    all_call_types = sorted(list(all_call_types))
    
    # Create rows for each payment type
    for payment_type, table in programme_tables.items():
        if payment_type == 'All_Payments':
            continue  # Skip the existing all payments, we'll create our own
            
        if len(table) == 0:
            continue
            
        total_row = table[table['Quarter'] == 'Total']
        if len(total_row) == 0:
            continue
            
        # === TOTAL AMOUNT ROW ===
        amount_row = {'Payment_Type': payment_type, 'Metric': 'Total Amount'}
        
        for call_type in all_call_types:
            amount_col = f'Total_Amount_{call_type}'
            amount_row[call_type] = total_row[amount_col].iloc[0] if amount_col in total_row.columns else 0
        
        amount_row['Total'] = total_row['Total_Amount'].iloc[0]
        summary_data.append(amount_row)
        
        # === VOBU/EFTA ROW ===
        vobu_efta_row = {'Payment_Type': payment_type, 'Metric': 'Out of Which VOBU/EFTA'}
        
        for call_type in all_call_types:
            vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
            vobu_efta_row[call_type] = total_row[vobu_efta_col].iloc[0] if vobu_efta_col in total_row.columns else 0
        
        vobu_efta_row['Total'] = total_row['VOBU_EFTA_Amount'].iloc[0]
        summary_data.append(vobu_efta_row)
        
        # === TRANSACTIONS ROW ===
        transactions_row = {'Payment_Type': payment_type, 'Metric': 'No of Transactions'}
        
        for call_type in all_call_types:
            transactions_col = f'No_of_Transactions_{call_type}'
            transactions_row[call_type] = total_row[transactions_col].iloc[0] if transactions_col in total_row.columns else 0
        
        transactions_row['Total'] = total_row['No_of_Transactions'].iloc[0]
        summary_data.append(transactions_row)
    
    # === CREATE OVERALL TOTALS ===
    if summary_data:
        # Get the All_Payments table data
        all_payments_table = programme_tables.get('All_Payments', pd.DataFrame())
        
        if len(all_payments_table) > 0:
            total_row = all_payments_table[all_payments_table['Quarter'] == 'Total']
            
            if len(total_row) > 0:
                # TOTAL AMOUNTS ACROSS ALL PAYMENT TYPES
                total_amount_row = {'Payment_Type': 'TOTAL ALL TYPES', 'Metric': 'Total Amount'}
                for call_type in all_call_types:
                    amount_col = f'Total_Amount_{call_type}'
                    total_amount_row[call_type] = total_row[amount_col].iloc[0] if amount_col in total_row.columns else 0
                total_amount_row['Total'] = total_row['Total_Amount'].iloc[0]
                summary_data.append(total_amount_row)
                
                # TOTAL VOBU/EFTA ACROSS ALL PAYMENT TYPES
                total_vobu_efta_row = {'Payment_Type': 'TOTAL ALL TYPES', 'Metric': 'Out of Which VOBU/EFTA'}
                for call_type in all_call_types:
                    vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
                    total_vobu_efta_row[call_type] = total_row[vobu_efta_col].iloc[0] if vobu_efta_col in total_row.columns else 0
                total_vobu_efta_row['Total'] = total_row['VOBU_EFTA_Amount'].iloc[0]
                summary_data.append(total_vobu_efta_row)
                
                # TOTAL TRANSACTIONS ACROSS ALL PAYMENT TYPES (deduplicated)
                total_transactions_row = {'Payment_Type': 'TOTAL ALL TYPES', 'Metric': 'No of Transactions'}
                for call_type in all_call_types:
                    transactions_col = f'No_of_Transactions_{call_type}'
                    total_transactions_row[call_type] = total_row[transactions_col].iloc[0] if transactions_col in total_row.columns else 0
                total_transactions_row['Total'] = total_row['No_of_Transactions'].iloc[0]
                summary_data.append(total_transactions_row)
    
    # Convert to DataFrame
    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        
        # Reorder columns
        column_order = ['Payment_Type', 'Metric'] + all_call_types + ['Total']
        summary_df = summary_df[column_order]
        
        return summary_df
    else:
        return pd.DataFrame()

def create_payment_type_comparison_table(results, programme):
    """
    Create a table showing just the totals for each payment type for easy comparison
    """
    
    if 'tables' not in results or programme not in results['tables']:
        return pd.DataFrame()
    
    programme_tables = results['tables'][programme]
    comparison_data = []
    
    for payment_type, table in programme_tables.items():
        if payment_type == 'All_Payments':
            continue
            
        if len(table) == 0:
            continue
            
        total_row = table[table['Quarter'] == 'Total']
        if len(total_row) == 0:
            continue
        
        comparison_row = {
            'Payment_Type': payment_type,
            'Total_Amount': total_row['Total_Amount'].iloc[0],
            'VOBU_EFTA_Amount': total_row['VOBU_EFTA_Amount'].iloc[0],
            'No_of_Transactions': total_row['No_of_Transactions'].iloc[0]
        }
        
        comparison_data.append(comparison_row)
    
    # Add overall total
    if comparison_data:
        all_payments_table = programme_tables.get('All_Payments', pd.DataFrame())
        if len(all_payments_table) > 0:
            total_row = all_payments_table[all_payments_table['Quarter'] == 'Total']
            if len(total_row) > 0:
                overall_row = {
                    'Payment_Type': 'TOTAL ALL TYPES',
                    'Total_Amount': total_row['Total_Amount'].iloc[0],
                    'VOBU_EFTA_Amount': total_row['VOBU_EFTA_Amount'].iloc[0],
                    'No_of_Transactions': total_row['No_of_Transactions'].iloc[0]
                }
                comparison_data.append(overall_row)
    
    return pd.DataFrame(comparison_data)

def list_available_tables(formatted_results):
    """
    List all available tables including summary options
    """
    print("=== AVAILABLE TABLES FOR GREAT_TABLES ===")
    
    if 'tables' not in formatted_results:
        print("No tables found")
        return
    
    for programme, tables in formatted_results['tables'].items():
        print(f"\n{programme} Programme:")
        for payment_type, df_table in tables.items():
            rows, cols = df_table.shape
            if payment_type == 'All_Payments':
                print(f"  - {payment_type}: {rows} rows x {cols} columns ⭐ SUMMARY TABLE")
            else:
                print(f"  - {payment_type}: {rows} rows x {cols} columns")
        
        print(f"\n  📊 Access functions available:")
        print(f"    # Individual payment types:")
        print(f"    get_great_table(results, '{programme}', 'Pre-financing', repeat_quarter=True)  # Recommended")
        print(f"    get_great_table_repeated(results, '{programme}', 'Pre-financing')  # Same as above")
        print(f"    get_great_table_grouped(results, '{programme}', 'Pre-financing')   # Excel visual style")
        print(f"    ")
        print(f"    # Summary tables:")
        print(f"    get_summary_table(results, '{programme}', repeat_quarter=True)  # All payment types")
        print(f"    create_comprehensive_summary_table(results, '{programme}')       # Alternative")
        print(f"    create_payment_type_comparison_table(results, '{programme}')     # Quick comparison")

def get_all_programme_tables(formatted_results, programme):
    """
    Get all tables for a specific programme as a dictionary
    """
    try:
        return formatted_results['tables'][programme]
    except KeyError:
        print(f"Programme not found: {programme}")
        available = list(formatted_results.get('tables', {}).keys())
        print(f"Available programmes: {available}")
        return {}

def combine_payment_types_table(formatted_results, programme):
    """
    Combine all payment types for a programme into one large table
    """
    programme_tables = get_all_programme_tables(formatted_results, programme)
    
    if not programme_tables:
        return pd.DataFrame()
    
    combined_tables = []
    
    for payment_type, df_table in programme_tables.items():
        if len(df_table) > 0:
            # Add a separator row if not the first table
            if len(combined_tables) > 0:
                separator_row = pd.DataFrame([{
                    'Metric': f'--- {payment_type} ---',
                    **{col: '' for col in df_table.columns if col != 'Metric'}
                }])
                combined_tables.append(separator_row)
            
            combined_tables.append(df_table)
    
    if combined_tables:
        return pd.concat(combined_tables, ignore_index=True)
    else:
        return pd.DataFrame()

# Example usage functions
def show_table_summary_for_great_tables(formatted_results):
    """Display summary of all generated tables for great_tables"""
    print("=== QUARTERLY TABLES SUMMARY FOR GREAT_TABLES ===")
    print(f"Reporting Period: {formatted_results['metadata']['reporting_year']}")
    print(f"Data Range: {formatted_results['metadata']['actual_date_range']['start'].strftime('%Y-%m-%d')} to {formatted_results['metadata']['actual_date_range']['end'].strftime('%Y-%m-%d')}")
    print(f"VOBU/EFTA Logic: Aggregates only EFTA and VOBU fund sources")
    
    if 'great_tables' in formatted_results:
        for programme, tables in formatted_results['great_tables'].items():
            print(f"\n{programme}:")
            for payment_type, table in tables.items():
                if len(table) > 0:
                    # Get total amounts from the 'Total Amount' row
                    total_amount_row = table[table['Metric'] == 'Total Amount']
                    vobu_efta_row = table[table['Metric'] == 'Out of Which VOBU/EFTA']
                    transactions_row = table[table['Metric'] == 'No of Transactions']
                    
                    if len(total_amount_row) > 0:
                        total_amount = total_amount_row['Total'].iloc[0]
                        vobu_efta_amount = vobu_efta_row['Total'].iloc[0] if len(vobu_efta_row) > 0 else 0
                        total_transactions = transactions_row['Total'].iloc[0] if len(transactions_row) > 0 else 0
                        print(f"  {payment_type}: €{total_amount:,.0f} total (€{vobu_efta_amount:,.0f} VOBU/EFTA) - {total_transactions} transactions")

# Updated access examples
# Backwards compatibility functions
def get_excel_format_table(formatted_results, programme, payment_type):
    """
    Backwards compatibility function - redirects to get_great_table
    """
    print("Note: get_excel_format_table is deprecated, use get_great_table for great_tables formatting")
    return get_great_table(formatted_results, programme, payment_type)

def show_table_summary(formatted_results):
    """
    Backwards compatibility function - redirects to show_table_summary_for_great_tables
    """
    return show_table_summary_for_great_tables(formatted_results)

# Updated access examples with summary tables
def print_usage_instructions():
    """
    Print instructions for using the generated tables including summary tables
    """
    print("""
=== HOW TO USE THE QUARTERLY TABLES (Two Format Options) ===

1. GENERATE TABLES:
   quarterly_tables = generate_all_quarterly_tables(df_paym, cutoff)

2. ACCESS INDIVIDUAL PAYMENT TYPE TABLES:
   
   # OPTION A: Repeated quarters (RECOMMENDED for great_tables)
   heu_prefinancing = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=True)
   heu_interim = get_great_table(quarterly_tables, 'HEU', 'Interim Payments', repeat_quarter=True)
   
   # OPTION B: Grouped quarters (Excel visual style)
   heu_prefinancing_grouped = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=False)
   
   # Shortcut functions:
   heu_prefinancing = get_great_table_repeated(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option A
   heu_prefinancing_grouped = get_great_table_grouped(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option B

3. ACCESS SUMMARY TABLES (ALL PAYMENT TYPES INCLUDING EXPERTS):
   
   # Summary table with repeated quarters (recommended for great_tables)
   heu_summary = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=True)
   
   # Summary table with grouped quarters (Excel visual style)
   heu_summary_grouped = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=False)
   
   # Alternative access
   heu_summary_alt = create_comprehensive_summary_table(quarterly_tables, 'HEU', repeat_quarter=True)
   
   # Quick comparison table (different format - just totals by payment type)
   heu_comparison = create_payment_type_comparison_table(quarterly_tables, 'HEU')

4. LIST ALL AVAILABLE TABLES:
   list_available_tables(quarterly_tables)

5. USE WITH GREAT_TABLES:
   from great_tables import GT
   
   # Individual payment type (using repeated format)
   gt_table = (
       GT(heu_prefinancing)
       .tab_header(title="HEU Pre-financing", subtitle="Q1 2025")
       .fmt_currency(columns=['ADG', 'COG', 'POC', 'STG', 'SYG', 'Total'], currency='EUR')
       .tab_row_group(label="2024Q1", rows=["2024Q1"])  # Can group by Quarter if using repeated format
   )

FORMAT OPTIONS:

OPTION A - Repeated Quarters (RECOMMENDED for great_tables):
Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
2024Q1  | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
2024Q1  | No of Transactions      | ... | ... | ... | ... | ... | ...
2024Q2  | Total Amount            | ... | ... | ... | ... | ... | ...
2024Q2  | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
2024Q2  | No of Transactions      | ... | ... | ... | ... | ... | ...

OPTION B - Grouped Quarters (Excel visual style):
Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
        | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
        | No of Transactions      | ... | ... | ... | ... | ... | ...
2024Q2  | Total Amount            | ... | ... | ... | ... | ... | ...
        | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
        | No of Transactions      | ... | ... | ... | ... | ... | ...

RECOMMENDATION:
✅ Use repeat_quarter=True (Option A) for great_tables - better for:
  - Data processing and filtering
  - Row grouping operations
  - Exporting to other formats
  - Table manipulation

Use repeat_quarter=False (Option B) for:
  - Pure visual presentation matching Excel
  - When you want minimal visual clutter
""")

def show_summary_examples():
    """
    Show examples of the different summary table options with correct format
    """
    print("""
=== SUMMARY TABLE EXAMPLES (Both Format Options) ===

# 1a. Summary table with repeated quarters (RECOMMENDED for great_tables)
heu_summary = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=True)
# Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
# 2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
# 2024Q1  | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
# 2024Q1  | No of Transactions      | ... | ... | ... | ... | ... | ...
# Total   | Total Amount            | ... | ... | ... | ... | ... | ...
# Total   | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
# Total   | No of Transactions      | ... | ... | ... | ... | ... | ...

# 1b. Summary table with grouped quarters (Excel visual style)
heu_summary_grouped = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=False)
# Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
# 2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
#         | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
#         | No of Transactions      | ... | ... | ... | ... | ... | ...
# Total   | Total Amount            | ... | ... | ... | ... | ... | ...
#         | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
#         | No of Transactions      | ... | ... | ... | ... | ... | ...

# 2. Quick comparison (different format - by payment type)
heu_comparison = create_payment_type_comparison_table(quarterly_tables, 'HEU')
# Payment_Type | Total_Amount | VOBU_EFTA_Amount | No_of_Transactions
# Pre-financing | 352,568,326 | 336,797,403 | 309
# Interim Payments | 185,710,686 | 175,320,845 | 115
# TOTAL ALL TYPES | 1,628,225,910 | 1,398,938,052 | 1,573

BOTH formats contain the same data, just different visual presentation!
""")

# If run directly, show instructions
if __name__ == "__main__":
    print_usage_instructions()
    print("\n" + "="*60 + "\n")
    show_summary_examples()


=== HOW TO USE THE QUARTERLY TABLES (Two Format Options) ===

1. GENERATE TABLES:
   quarterly_tables = generate_all_quarterly_tables(df_paym, cutoff)

2. ACCESS INDIVIDUAL PAYMENT TYPE TABLES:

   # OPTION A: Repeated quarters (RECOMMENDED for great_tables)
   heu_prefinancing = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=True)
   heu_interim = get_great_table(quarterly_tables, 'HEU', 'Interim Payments', repeat_quarter=True)

   # OPTION B: Grouped quarters (Excel visual style)
   heu_prefinancing_grouped = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=False)

   # Shortcut functions:
   heu_prefinancing = get_great_table_repeated(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option A
   heu_prefinancing_grouped = get_great_table_grouped(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option B

3. ACCESS SUMMARY TABLES (ALL PAYMENT TYPES INCLUDING EXPERTS):

   # Summary table with repeated quarters (recommended for 

In [41]:
# Example usage:
quarterly_tables = generate_all_quarterly_tables(df_paym, cutoff )


Starting quarterly table generation for great_tables...
Using provided cutoff date: 2025-06-01 00:00:00
=== QUARTERLY PAYMENT TABLES GENERATION ===
Cutoff date: 2025-06-01 00:00:00
Reporting year: 2025
Expected scope: 2025-01-01 00:00:00 to 2025-06-30 00:00:00
Note: Assuming df_paym is already filtered for this scope
Found call type column: call_type
✓ All required columns present
Working dataset: 4890 rows
Actual date range: 2025-01-16 00:00:00 to 2025-06-02 00:00:00
Quarters found: ['2025Q1', '2025Q2']
Call types found: ['ADG', 'COG', 'EXPERTS', 'POC', 'STG', 'SYG']
Programmes found: ['HEU' 'H2020']

=== Processing HEU ===
  Creating table for: Pre-financing
  Creating table for: Interim Payments
  Creating table for: Experts and Support
  Creating table for: Final Payments
  Creating overall summary table

=== Processing H2020 ===
  Creating table for: Interim Payments
  Creating table for: Final Payments
  Creating overall summary table
  Formatting for great_tables - Call types: [

In [42]:
heu_prefinancing = get_great_table(quarterly_tables, 'HEU', 'Pre-financing')
heu_prefinancing

,Quarter,Metric,ADG,COG,POC,STG,SYG,Total
0,1Q2025,Total Amount,60065912.4,8.454135e+07,5062500.0,1.247404e+08,79357691.7,3.537678e+08
1,1Q2025,Out of Which VOBU/EFTA,60065912.4,8.072030e+07,4965000.0,1.128881e+08,79357691.7,3.379970e+08
2,1Q2025,No of Transactions,37.0,6.600000e+01,45.0,1.390000e+02,23.0,3.100000e+02
3,2Q2025,Total Amount,5099243.4,1.391124e+08,5962500.0,2.401659e+07,46452946.4,2.206437e+08
4,2Q2025,Out of Which VOBU/EFTA,5099243.4,1.382231e+08,3640000.0,1.629071e+07,45457190.4,2.087103e+08
5,2Q2025,No of Transactions,3.0,1.150000e+02,55.0,2.600000e+01,14.0,2.130000e+02
6,Total,Total Amount,65165155.8,2.236538e+08,11025000.0,1.487570e+08,125810638.1,5.744116e+08
7,Total,Out of Which VOBU/EFTA,65165155.8,2.189434e+08,8605000.0,1.291788e+08,124814882.1,5.467072e+08
8,Total,No of Transactions,40.0,1.810000e+02,100.0,1.650000e+02,37.0,5.230000e+02


In [43]:
heu_ip = get_great_table(quarterly_tables, 'HEU', 'Interim Payments')
heu_ip

,Quarter,Metric,ADG,COG,STG,SYG,Total
0,1Q2025,Total Amount,5671359.26,8931795.08,30656937.93,22497012.74,6.775711e+07
1,1Q2025,Out of Which VOBU/EFTA,5671359.26,8432021.08,25796245.16,21207250.51,6.110688e+07
2,1Q2025,No of Transactions,10.00,18.00,95.00,15.00,1.380000e+02
3,2Q2025,Total Amount,15793008.09,22838494.46,38504633.62,8040321.52,8.517646e+07
4,2Q2025,Out of Which VOBU/EFTA,15793008.09,22838494.46,31024571.32,8040321.52,7.769640e+07
5,2Q2025,No of Transactions,26.00,47.00,127.00,5.00,2.050000e+02
6,Total,Total Amount,21464367.35,31770289.54,69161571.55,30537334.26,1.529336e+08
7,Total,Out of Which VOBU/EFTA,21464367.35,31270515.54,56820816.48,29247572.03,1.388033e+08
8,Total,No of Transactions,36.00,65.00,222.00,20.00,3.430000e+02


In [44]:
heu_fp = get_great_table(quarterly_tables, 'HEU', 'Final Payments')
heu_fp

,Quarter,Metric,POC,Total
0,1Q2025,Total Amount,1632000.00,1632000.00
1,1Q2025,Out of Which VOBU/EFTA,1541863.75,1541863.75
2,1Q2025,No of Transactions,58.00,58.00
3,2Q2025,Total Amount,1140000.00,1140000.00
4,2Q2025,Out of Which VOBU/EFTA,1120000.00,1120000.00
5,2Q2025,No of Transactions,39.00,39.00
6,Total,Total Amount,2772000.00,2772000.00
7,Total,Out of Which VOBU/EFTA,2661863.75,2661863.75
8,Total,No of Transactions,97.00,97.00


In [45]:
heu_expt = get_great_table(quarterly_tables, 'HEU', 'Experts and Support')
heu_expt

,Quarter,Metric,EXPERTS,Total
0,1Q2025,Total Amount,3986096.99,3986096.99
1,1Q2025,Out of Which VOBU/EFTA,3941406.67,3941406.67
2,1Q2025,No of Transactions,1493.00,1493.00
3,2Q2025,Total Amount,3974216.64,3974216.64
4,2Q2025,Out of Which VOBU/EFTA,3974216.64,3974216.64
5,2Q2025,No of Transactions,1313.00,1313.00
6,Total,Total Amount,7960313.63,7960313.63
7,Total,Out of Which VOBU/EFTA,7915623.31,7915623.31
8,Total,No of Transactions,2806.00,2806.00


In [46]:
heu_all = get_great_table(quarterly_tables, 'HEU', 'All_Payments')
heu_all

,Quarter,Metric,ADG,COG,EXPERTS,POC,STG,SYG,Total
0,1Q2025,Total Amount,65737271.66,9.347314e+07,3986096.99,6694500.00,1.553973e+08,1.018547e+08,4.271430e+08
1,1Q2025,Out of Which VOBU/EFTA,65737271.66,8.915232e+07,3941406.67,6506863.75,1.386843e+08,1.005649e+08,4.045871e+08
2,1Q2025,No of Transactions,47.00,8.400000e+01,1493.00,103.00,2.340000e+02,3.800000e+01,1.999000e+03
3,2Q2025,Total Amount,20892251.49,1.619509e+08,3974216.64,7102500.00,6.252123e+07,5.449327e+07,3.109344e+08
4,2Q2025,Out of Which VOBU/EFTA,20892251.49,1.610616e+08,3974216.64,4760000.00,4.731529e+07,5.349751e+07,2.915009e+08
5,2Q2025,No of Transactions,29.00,1.620000e+02,1313.00,94.00,1.530000e+02,1.900000e+01,1.770000e+03
6,Total,Total Amount,86629523.15,2.554241e+08,7960313.63,13797000.00,2.179186e+08,1.563480e+08,7.380775e+08
7,Total,Out of Which VOBU/EFTA,86629523.15,2.502140e+08,7915623.31,11266863.75,1.859996e+08,1.540625e+08,6.960880e+08
8,Total,No of Transactions,76.00,2.460000e+02,2806.00,197.00,3.870000e+02,5.700000e+01,3.769000e+03


In [47]:
vars_all = fetch_vars_for_report(report, db_path)
heu_vars_data = vars_all.get('table_2a_HE')
h2020_vars_data = vars_all.get('table_2a_H2020')

heu_total_appropriations = next(
    (item['Available_Payment_Appropriations'] for item in heu_vars_data 
     if item['Budget Address Type'] == 'Total'), 
    None
)

heu_total_appropriations_expt = next(
    (item['Available_Payment_Appropriations'] for item in heu_vars_data 
     if item['Budget Address Type'] == 'Experts'), 
    None
)

h2020_total_appropriations = next(
    (item['Available_Payment_Appropriations'] for item in h2020_vars_data 
     if item['Budget Address Type'] == 'Total'), 
    None
)


In [ ]:
db_path = "database/reporting.db"
DB_PATH = Path("database/reporting.db")
report = 'Quarterly_Report'
report_params = load_report_params(report_name=report, db_path=DB_PATH )
TTP_gross = report_params.get("TTP_GROSS_HISTORY")
TTP_gross_H2020 = TTP_gross.get('H2020')


In [ ]:
import pandas as pd
from pathlib import Path

# =============================================================================
# CLEAN TTP CALCULATION FUNCTIONS
# =============================================================================

def calculate_current_ttp_metrics(df_paym, cutoff):
    """
    Calculate current TTP metrics from df_paym data
    """
    # Filter data up to cutoff and deduplicate by Pay Payment Key
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]

    df_filtered = df_paym[
        df_paym['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
    ].copy()
    df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
    
    # Convert to numeric
    df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
    df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
    df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')

    # Filter out negative TTP_NET values
    df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]
    
    results = {}
    
    # Calculate by Programme and Payment Type
    for programme in ['H2020', 'HEU']:
        prog_data = df_unique[df_unique['Programme'] == programme]
        if len(prog_data) == 0:
            continue
            
        results[programme] = {}
        
        # Overall programme metrics
        prog_valid = prog_data[prog_data['v_payment_in_time'].notna()]
        results[programme]['overall'] = {
            'avg_ttp_net': prog_data['v_TTP_NET'].mean(),
            'avg_ttp_gross': prog_data['v_TTP_GROSS'].mean(),
            'on_time_pct': prog_data['v_payment_in_time'].sum() / len(prog_valid) if len(prog_valid) > 0 else 0
        }
        
        # By payment type - using correct short form values from v_payment_type
        payment_types = ['IP', 'FP', 'EXPERTS', 'PF']  # Short form values
        
        for payment_type in payment_types:
            pt_data = prog_data[prog_data['v_payment_type'] == payment_type]
            if len(pt_data) > 0:
                pt_valid = pt_data[pt_data['v_payment_in_time'].notna()]
                results[programme][payment_type] = {
                    'avg_ttp_net': pt_data['v_TTP_NET'].mean(),
                    'avg_ttp_gross': pt_data['v_TTP_GROSS'].mean(),
                    'on_time_pct': pt_data['v_payment_in_time'].sum() / len(pt_valid) if len(pt_valid) > 0 else 0
                }
    
    # Overall total
    total_valid = df_unique[df_unique['v_payment_in_time'].notna()]
    results['TOTAL'] = {
        'avg_ttp_net': df_unique['v_TTP_NET'].mean(),
        'avg_ttp_gross': df_unique['v_TTP_GROSS'].mean(),
        'on_time_pct': df_unique['v_payment_in_time'].sum() / len(total_valid) if len(total_valid) > 0 else 0
    }
    
    return results

def load_historical_ttp_data(report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Load historical TTP data from database
    """
    DB_PATH = Path(db_path)
    report_params = load_report_params(report_name=report_name, db_path=DB_PATH)
    
    return {
        "TTP_NET_HISTORY": report_params.get("TTP_NET_HISTORY"),
        "TTP_GROSS_HISTORY": report_params.get("TTP_GROSS_HISTORY"),
        "PAYMENTS_ON_TIME_HISTORY": report_params.get("PAYMENTS_ON_TIME_HISTORY")
    }

def create_ttp_comparison_table(df_paym, cutoff, historical_data):
    """
    Create TTP comparison table matching the image structure
    """
    # Calculate current metrics
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    
    # Determine labels based on cutoff
    cutoff_date = pd.to_datetime(cutoff)
    current_year = cutoff_date.year
    current_label = f"{current_year}-YTD"
    historical_label = f"Dec {current_year - 1}"
    
    # Build comparison data
    comparison_data = []
    
    # H2020 section
    h2020_current = current_metrics.get('H2020', {})
    
    # H2020 - Interim Payments (IP)
    current_ip = h2020_current.get('IP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Interim Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_ip['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("IP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_ip['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("IP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_ip['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('IP', 0)*100:.2f}%"
    })
    
    # H2020 - Final Payments (FP)
    current_fp = h2020_current.get('FP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Final Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_fp['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("FP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_fp['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("FP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_fp['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('FP', 0)*100:.2f}%"
    })
    
    # H2020 - Experts Payments (EXPERTS)
    current_exp = h2020_current.get('EXPERTS', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Experts Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_exp['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("Experts", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_exp['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("Experts", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_exp['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('Experts', 0)*100:.2f}%"
    })
    
    # H2020 - Overall
    current_h2020 = h2020_current.get('overall', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'H2020',
        f'Average Net Time to Pay (in days) {current_label}': round(current_h2020['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("H2020", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_h2020['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("H2020", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_h2020['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('H2020', 0)*100:.2f}%"
    })
    
    # HEU section
    heu_current = current_metrics.get('HEU', {})
    
    # HEU - Prefinancing Payments (PF)
    current_pf = heu_current.get('PF', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Prefinancing Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_pf['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("PF", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_pf['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("PF", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_pf['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('PF', 0)*100:.2f}%"
    })
    
    # HEU - Interim Payments (IP)
    current_ip_heu = heu_current.get('IP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Interim Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_ip_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("IP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_ip_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("IP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_ip_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('IP', 0)*100:.2f}%"
    })
    
    # HEU - Final Payments (FP)
    current_fp_heu = heu_current.get('FP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Final Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_fp_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("FP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_fp_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("FP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_fp_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('FP', 0)*100:.2f}%"
    })
    
    # HEU - Experts Payment (EXPERTS)
    current_exp_heu = heu_current.get('EXPERTS', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Experts Payment',
        f'Average Net Time to Pay (in days) {current_label}': round(current_exp_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("Experts", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_exp_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("Experts", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_exp_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('Experts', 0)*100:.2f}%"
    })
    
    # HEU - Overall
    current_heu = heu_current.get('overall', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'HEU',
        f'Average Net Time to Pay (in days) {current_label}': round(current_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("HEU", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("HEU", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('HEU', 0)*100:.2f}%"
    })
    
    # TOTAL row
    current_total = current_metrics.get('TOTAL', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'TOTAL',
        f'Average Net Time to Pay (in days) {current_label}': round(current_total['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["ALL"].get("TOTAL", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_total['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["ALL"].get("TOTAL", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_total['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['ALL'].get('TOTAL', 0)*100:.2f}%"
    })
    
    return pd.DataFrame(comparison_data)

def create_ttp_effectiveness_table(df_paym, cutoff, historical_data, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Create TTP effectiveness and efficiency indicators table
    """
    # Calculate current metrics
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    
    # Determine labels based on cutoff using get_scope_start_end
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]
    current_month = pd.to_datetime(last_valid_date).strftime('%b-%y')
    
    cutoff_date = pd.to_datetime(cutoff)
    current_year = cutoff_date.year
    historical_label = f"Dec-{str(current_year-1)[-2:]}"
    
    # Load database parameters for admin and expert meetings
    from pathlib import Path
    DB_PATH = Path(db_path)
    report_params = load_report_params(report_name=report_name, db_path=DB_PATH)
    
    admin_eff = report_params.get("Administrative_expenditure_Effectiveness", {})
    admin_ttp = report_params.get("Administrative_expenditure_ttp", {})
    expt_meet_eff = report_params.get("Expert_meetings_Effectiveness", {})
    expt_meet_ttp = report_params.get("Expert_meetings_ttp", {})
    
    # Build effectiveness data
    effectiveness_data = []
    
    # Get current metrics for calculations
    h2020_current = current_metrics.get('H2020', {})
    heu_current = current_metrics.get('HEU', {})
    
    # Research grants - Interim Payments - H2020
    h2020_ip = h2020_current.get('IP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Interim Payments - H2020',
        current_month: f"{h2020_ip*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('IP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Research grants - Final Payments - H2020
    h2020_fp = h2020_current.get('FP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Final Payments - H2020',
        current_month: f"{h2020_fp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('FP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Experts with Appointment Letters H2020
    h2020_exp = h2020_current.get('EXPERTS', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Experts with Appointment Letters H2020',
        current_month: f"{h2020_exp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('Experts', 0)*100:.2f}%",
        'Target': '95% in 30 days'
    })
    
    # Research grants - Pre-financings HEU
    heu_pf = heu_current.get('PF', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Pre-financings HEU',
        current_month: f"{heu_pf*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('PF', 0)*100:.2f}%",
        'Target': '95% in 30 days'
    })
    
    # Research grants - Interim Payments HEU
    heu_ip = heu_current.get('IP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Interim Payments HEU',
        current_month: f"{heu_ip*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('IP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Research grants - Final Payments HEU
    heu_fp = heu_current.get('FP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Final Payments HEU',
        current_month: f"{heu_fp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('FP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Administrative expenditure (from database)
    admin_current = admin_eff.get("Current", 0)
    admin_old = admin_eff.get("Old", 0)
    admin_target = admin_eff.get("Target", "99% in 30 days")
    
    # Format admin values - handle both percentage (0.985) and already formatted (98.5%) values
    if isinstance(admin_current, (int, float)) and admin_current <= 1:
        admin_current_str = f"{admin_current*100:.2f}%"
    else:
        admin_current_str = str(admin_current) if admin_current != "na" else "n/a"
        
    if isinstance(admin_old, (int, float)) and admin_old <= 1:
        admin_old_str = f"{admin_old*100:.2f}%"
    else:
        admin_old_str = str(admin_old) if admin_old != "na" else "n/a"
    
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Administrative expenditure',
        current_month: admin_current_str,
        historical_label: admin_old_str,
        'Target': admin_target
    })
    
    # Experts with Appointment Letters HEU
    heu_exp = heu_current.get('EXPERTS', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Experts with Appointment Letters HEU',
        current_month: f"{heu_exp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('Experts', 0)*100:.2f}%",
        'Target': '95% in 30 days'
    })
    
    # Expert meetings PMO (from database)
    expt_current = expt_meet_eff.get("Current", "na")
    expt_old = expt_meet_eff.get("Old", "na")
    expt_target = expt_meet_eff.get("Target", "n/a")
    
    # Format expert values - handle both percentage and string values
    if isinstance(expt_current, (int, float)) and expt_current <= 1:
        expt_current_str = f"{expt_current*100:.2f}%"
    else:
        expt_current_str = str(expt_current) if expt_current != "na" else "n/a"
        
    if isinstance(expt_old, (int, float)) and expt_old <= 1:
        expt_old_str = f"{expt_old*100:.2f}%"
    else:
        expt_old_str = str(expt_old) if expt_old != "na" else "n/a"
    
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Expert meetings PMO',
        current_month: expt_current_str,
        historical_label: expt_old_str,
        'Target': expt_target
    })
    
    return pd.DataFrame(effectiveness_data)



def create_ttp_days_table(df_paym, cutoff, historical_data, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Create Time to Pay: Average number of days (H2020 - HEU) table
    """
    # Calculate current metrics
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    
    # Load database parameters for admin and expert meetings
    from pathlib import Path
    DB_PATH = Path(db_path)
    report_params = load_report_params(report_name=report_name, db_path=DB_PATH)
    
    admin_eff = report_params.get("Administrative_expenditure_Effectiveness", {})
    admin_ttp = report_params.get("Administrative_expenditure_ttp", {})
    expt_meet_eff = report_params.get("Expert_meetings_Effectiveness", {})
    expt_meet_ttp = report_params.get("Expert_meetings_ttp", {})
    
    # Build days data
    days_data = []
    
    # Get current metrics for calculations
    h2020_current = current_metrics.get('H2020', {})
    heu_current = current_metrics.get('HEU', {})
    
    # Research grants (days) - Interim Payments- H2020
    h2020_ip = h2020_current.get('IP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Interim Payments- H2020',
        'NET': round(h2020_ip.get('avg_ttp_net', 0), 1),
        'GROSS': round(h2020_ip.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Research grants (days) - Final Payments- H2020
    h2020_fp = h2020_current.get('FP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Final Payments- H2020',
        'NET': round(h2020_fp.get('avg_ttp_net', 0), 1),
        'GROSS': round(h2020_fp.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Experts with Appointment Letters (days) H2020
    h2020_exp = h2020_current.get('EXPERTS', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Experts with Appointment Letters (days) H2020',
        'NET': round(h2020_exp.get('avg_ttp_net', 0), 1),
        'GROSS': round(h2020_exp.get('avg_ttp_gross', 0), 1),
        'Target': '30 days'
    })
    
    # Research grants (days) - Pre-financings - HEU
    heu_pf = heu_current.get('PF', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Pre-financings - HEU',
        'NET': round(heu_pf.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_pf.get('avg_ttp_gross', 0), 1),
        'Target': '30 days'
    })
    
    # Research grants (days) - Interim Payments- HEU
    heu_ip = heu_current.get('IP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Interim Payments- HEU',
        'NET': round(heu_ip.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_ip.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Research grants (days) - Final Payments- HEU
    heu_fp = heu_current.get('FP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Final Payments- HEU',
        'NET': round(heu_fp.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_fp.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Expert with Appointment Letter (days) HEU
    heu_exp = heu_current.get('EXPERTS', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Expert with Appointment Letter (days) HEU',
        'NET': round(heu_exp.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_exp.get('avg_ttp_gross', 0), 1),
        'Target': '30 days'
    })
    
    # Expert meetings (PMO) (days) - from database
    expt_net_current = expt_meet_ttp.get("Current", "na")
    expt_gross_current = expt_meet_ttp.get("Current", "na")  # Assuming same for both NET and GROSS
    
    # Format expert meeting values
    if isinstance(expt_net_current, (int, float)):
        expt_net_str = str(round(expt_net_current, 1))
        expt_gross_str = str(round(expt_gross_current, 1))
    else:
        expt_net_str = "n/a"
        expt_gross_str = "n/a"
    
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Expert meetings (PMO) (days)',
        'NET': expt_net_str,
        'GROSS': expt_gross_str,
        'Target': '30 days'
    })
    
    # Administrative expenditure (days) - from database
    admin_net_current = admin_ttp.get("Current", 0)
    admin_gross_current = admin_ttp.get("Current", 0)  # Assuming same for both NET and GROSS
    
    # Format admin values
    if isinstance(admin_net_current, (int, float)):
        admin_net_str = str(round(admin_net_current, 1))
        admin_gross_str = str(round(admin_gross_current, 1))
    else:
        admin_net_str = "n/a"
        admin_gross_str = "n/a"
    
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Administrative expenditure (days)',
        'NET': admin_net_str,
        'GROSS': admin_gross_str,
        'Target': '30 days'
    })
    
    return pd.DataFrame(days_data)

def generate_all_ttp_tables(df_paym, cutoff, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Generate all three TTP tables - comparison table, effectiveness table, and days table
    
    Usage in Jupyter:
    comparison_table, effectiveness_table, days_table = generate_all_ttp_tables(df_paym, cutoff)
    """
    # Load historical data
    historical_data = load_historical_ttp_data(report_name=report_name, db_path=db_path)
    
    # Create all three tables
    comparison_table = create_ttp_comparison_table(df_paym, cutoff, historical_data)
    effectiveness_table = create_ttp_effectiveness_table(df_paym, cutoff, historical_data, report_name, db_path)
    days_table = create_ttp_days_table(df_paym, cutoff, historical_data, report_name, db_path)
    
    return comparison_table, effectiveness_table, days_table

# =============================================================================
# UPDATED MAIN FUNCTION - USE THIS IN JUPYTER
# =============================================================================

def generate_ttp_tables(df_paym, cutoff, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Main function to generate all TTP tables
    
    Usage in Jupyter:
    comparison_table, effectiveness_table, days_table = generate_ttp_tables(df_paym, cutoff)
    
    # Display all three tables
    print("TTP Comparison Table:")
    display(comparison_table)
    
    print("\nEffectiveness and Efficiency Indicators:")
    display(effectiveness_table)
    
    print("\nTime to Pay: Average number of days:")
    display(days_table)
    """
    return generate_all_ttp_tables(df_paym, cutoff, report_name, db_path)

In [ ]:
# =============================================================================
# USAGE IN JUPYTER NOTEBOOK
# =============================================================================


# Use this in your Jupyter notebook:
comparison_table, effectiveness_table, ttp_days_table = generate_ttp_tables(df_paym, cutoff)
comparison_table



In [ ]:
effectiveness_table

In [ ]:
ttp_days_table

In [ ]:
import pandas as pd
from pathlib import Path

# =============================================================================
# TABLE GENERATION FUNCTIONS
# =============================================================================

def calculate_current_ttp_metrics(df_paym, cutoff):
    """
    Calculate current TTP metrics from df_paym data, filtering out negative v_TTP_NET
    """
    # Filter data up to cutoff and deduplicate by Pay Payment Key
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]

    df_filtered = df_paym[
        df_paym['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
    ].copy()
    df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
    
    # Convert to numeric and filter negative v_TTP_NET
    df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
    df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
    df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')
    df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]
    
    results = {}
    
    # Calculate by Programme and Payment Type
    for programme in ['H2020', 'HEU']:
        prog_data = df_unique[df_unique['Programme'] == programme]
        if len(prog_data) == 0:
            continue
            
        results[programme] = {}
        
        # Overall programme metrics
        prog_valid = prog_data[prog_data['v_payment_in_time'].notna()]
        results[programme]['overall'] = {
            'avg_ttp_net': prog_data['v_TTP_NET'].mean(),
            'avg_ttp_gross': prog_data['v_TTP_GROSS'].mean(),
            'on_time_pct': prog_data['v_payment_in_time'].sum() / len(prog_valid) if len(prog_valid) > 0 else 0
        }
        
        # By payment type - using correct short form values from v_payment_type
        payment_types = ['IP', 'FP', 'EXPERTS', 'PF']  # Short form values
        for payment_type in payment_types:
            pt_data = prog_data[prog_data['v_payment_type'] == payment_type]
            if len(pt_data) > 0:
                pt_valid = pt_data[pt_data['v_payment_in_time'].notna()]
                results[programme][payment_type] = {
                    'avg_ttp_net': pt_data['v_TTP_NET'].mean(),
                    'avg_ttp_gross': pt_data['v_TTP_GROSS'].mean(),
                    'on_time_pct': pt_data['v_payment_in_time'].sum() / len(pt_valid) if len(pt_valid) > 0 else 0
                }
    
    # Overall total
    total_valid = df_unique[df_unique['v_payment_in_time'].notna()]
    results['TOTAL'] = {
        'avg_ttp_net': df_unique['v_TTP_NET'].mean(),
        'avg_ttp_gross': df_unique['v_TTP_GROSS'].mean(),
        'on_time_pct': df_unique['v_payment_in_time'].sum() / len(total_valid) if len(total_valid) > 0 else 0
    }
    
    return results

def create_quarterly_ttp_table(df_paym, cutoff, programme, payment_type):
    """
    Create a quarterly TTP table for a specific programme and payment type
    Returns table, programme, payment_type, and a flag indicating if the table is empty
    """
    # Filter data
    df_filtered = df_paym[
        (df_paym['Pay Document Date (dd/mm/yyyy)'] <= pd.to_datetime(cutoff)) &
        (df_paym['Programme'] == programme) &
        (df_paym['v_payment_type'] == payment_type)
    ].copy()
    
    # If no data after filtering, return an empty table with a flag
    if df_filtered.empty:
        empty_table = pd.DataFrame(columns=['Quarter', 'ADG', 'COG', 'POC', 'STG', 'SYG', 'Total'])
        return empty_table, programme, payment_type, True
    
    # Convert to numeric and filter negative TTP_NET
    df_filtered['v_TTP_NET'] = pd.to_numeric(df_filtered['v_TTP_NET'], errors='coerce')
    df_filtered = df_filtered[df_filtered['v_TTP_NET'] >= 0]
    
    # If all TTP_NET values are filtered out, return an empty table
    if df_filtered.empty:
        empty_table = pd.DataFrame(columns=['Quarter', 'ADG', 'COG', 'POC', 'STG', 'SYG', 'Total'])
        return empty_table, programme, payment_type, True
    
    # Extract quarter from date
    df_filtered['Quarter'] = pd.to_datetime(df_filtered['Pay Document Date (dd/mm/yyyy)']).dt.to_period('Q').astype(str)
    
    # Use call_type as CallType (based on logs)
    if 'call_type' not in df_filtered.columns:
        df_filtered['call_type'] = 'Default'  # Placeholder if call_type is missing
    df_filtered['CallType'] = df_filtered['call_type']
    
    # Aggregate by Quarter and CallType (using v_TTP_NET mean as metric)
    quarterly_data = df_filtered.groupby(['Quarter', 'CallType']).agg({
        'v_TTP_NET': 'mean'
    }).round(1).unstack(fill_value=0)
    
    # Rename columns to match call types from logs (ADG, COG, etc.)
    quarterly_data.columns = [f'{col[1]}' for col in quarterly_data.columns]
    
    # Add total row (mean across quarters for each call type)
    total_row = quarterly_data.mean().rename('Total')
    quarterly_table = pd.concat([quarterly_data, total_row.to_frame().T])
    
    # Simplify index to only include Quarter
    quarterly_table.index = quarterly_table.index.droplevel([1, 2]) if isinstance(quarterly_table.index, pd.MultiIndex) else quarterly_table.index
    
    return quarterly_table, programme, payment_type, False

def generate_quarterly_tables(df_paym, cutoff):
    """
    Generate quarterly TTP tables for each programme and payment type
    Includes a flag to indicate empty tables
    """
    tables = {}
    payment_types = ['IP', 'FP', 'EXPERTS', 'PF']  # Short form values
    programs = ['H2020', 'HEU']

    for prog in programs:
        for pt in payment_types:
            table, program, payment_type, is_empty = create_quarterly_ttp_table(df_paym, cutoff, prog, pt)
            tables[f'{prog}_{pt}_table'] = {
                'table': table,
                'programme': program,
                'payment_type': payment_type,
                'is_empty': is_empty
            }

    return tables

# =============================================================================
# MAIN USAGE FOR TABLES
# =============================================================================

def main_tables(df_paym, cutoff):
    """
    Main function to generate and display quarterly TTP tables
    """
    # Generate tables
    quarterly_tables = generate_quarterly_tables(df_paym, cutoff)
    
    # Display tables (in Jupyter Notebook)
    for key, data in quarterly_tables.items():
        table = data['table']
        program = data['programme']
        payment_type = data['payment_type']
        is_empty = data['is_empty']
        print(f"\n{key} (Programme: {program}, Payment Type: {payment_type}, Empty: {is_empty}):")
        display(table)
    
    return quarterly_tables

# Example usage (uncomment to run)
# quarterly_tables = main_tables(df_paym, cutoff)

In [ ]:


# Example usage (uncomment to run)
quarterly_tables = main_tables(df_paym, cutoff)

In [ ]:
import pandas as pd
import altair as alt
import time
import warnings
from IPython.display import display, clear_output



# Suppress the Altair FutureWarning about convert_dtype
warnings.filterwarnings("ignore", message=".*convert_dtype.*", category=FutureWarning)

def rolling_ttp(df,programme,typeofpayment):
    
        start, end  = get_scope_start_end(cutoff)
        last_month = int(end.month)
        i = 1
        moving_avg_ttp = []
        months = []

        quarter_dates = get_scope_start_end(cutoff=cutoff)
        last_valid_date = quarter_dates[1]

        df_filtered = df[
                df['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
        ].copy()
        df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
        
        # Convert to numeric and filter negative v_TTP_NET
        df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
        df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
        df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')
        df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]

        while i <= last_month : 
                df_test =  df_unique.loc[( df_unique['Programme'] == programme) & ( df_unique['v_payment_type'] == typeofpayment) & ( df_unique['Month']<= i )]    
                mean = round(df_test['TTP_NET'].mean(),1)
                moving_avg_ttp.insert(i,mean)
                months.insert(i,i)
                i+=1

        d = {'Month': months, 'TTP': moving_avg_ttp}
        df_mov_ttp = pd.DataFrame(data=d)
        return df_mov_ttp 

def avg_ttp(df,programme,typeofpayment):
    
        start, end  = get_scope_start_end(cutoff)
        last_month = int(end.month)
        i = 1
        moving_avg_ttp = []
        months = []

        quarter_dates = get_scope_start_end(cutoff=cutoff)
        last_valid_date = quarter_dates[1]

        df_filtered = df[
                df['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
        ].copy()
        df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
        
        # Convert to numeric and filter negative v_TTP_NET
        df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
        df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
        df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')
        df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]

        pivot_ttp_month =  df_unique.pivot_table( index= df_unique[["Month"]], values= df_unique[['TTP_NET']],fill_value=0,aggfunc='mean')
        pivot_ttp_month['TTP_NET'] = pivot_ttp_month['TTP_NET'].round(1)
        #pivot_ttp_month.columns =  pivot_ttp_month.columns.droplevel()
        pivot_ttp_month.reset_index(inplace = True)

        return pivot_ttp_month


# =============================================================================
# CHART GENERATION FUNCTIONS
# =============================================================================

def chart_machine_ttp(df, paymentType, prog, avg, rollingAvg):
    """Clean version with all positioning and styling issues fixed"""
    
    # Time limits
    time_limits = {
        'PF': 30, 'EXPERTS': 30, 'IP': 90, 'FP': 90
    }
    time_limit = time_limits.get(paymentType, 90)
    
    # Clean data
    df_clean = df.copy()
    df_clean['Month'] = pd.to_numeric(df_clean['Month'], errors='coerce')
    df_clean['TTP_NET'] = pd.to_numeric(df_clean['TTP_NET'], errors='coerce')
    df_clean = df_clean.dropna(subset=['Month', 'TTP_NET'])
    
    rollingAvg_clean = rollingAvg.copy()
    rollingAvg_clean['Month'] = pd.to_numeric(rollingAvg_clean['Month'], errors='coerce')
    rollingAvg_clean['TTP'] = pd.to_numeric(rollingAvg_clean['TTP'], errors='coerce')
    rollingAvg_clean = rollingAvg_clean.dropna()
    
    if df_clean.empty:
        return alt.Chart().mark_text(text="No data available")
    
    # Calculate Y-axis range - extend if values are close to limits
    max_value = max(
        df_clean['TTP_NET'].max() if not df_clean.empty else 0,
        rollingAvg_clean['TTP'].max() if not rollingAvg_clean.empty else 0,
        time_limit
    )
    
    # Extend Y-axis if monthly average is close to time limit (within 10 days)
    if abs(avg - time_limit) <= 10:
        y_max = max_value * 1.2  # Extend by 20%
    else:
        y_max = max_value * 1.1  # Normal 10% extension
    
    # Get last month with data for triangle positioning
    last_month_with_data = df_clean['Month'].max() if not df_clean.empty else 1
    start, end = get_scope_start_end(cutoff)
    year = int(end.year)
    
    # Get rolling average value for triangle positioning - position exactly on the line
    last_rolling_value = rollingAvg_clean[
        rollingAvg_clean['Month'] == last_month_with_data
    ]['TTP'].values
    triangle_y_pos = last_rolling_value[0] if len(last_rolling_value) > 0 else avg
    
    # Create triangle annotation data positioned on the rolling average line
    triangle_data = pd.DataFrame({
        'Month': [last_month_with_data],
        'TTP': [triangle_y_pos],  # Exact position on rolling average line
        'Triangle': ['⯆'],  # Down-pointing arrow
        'Comment': [f'{prog} Average {year} = {avg}']
    })
    
    # Main bars with darker color for better label visibility
    bars = alt.Chart(df_clean).mark_bar(
        opacity=0.8,
        color='#4682B4'  # Darker steel blue for better contrast
    ).encode(
        x=alt.X('Month:O', title='Month'),
        y=alt.Y('TTP_NET:Q', title='Days', scale=alt.Scale(domain=[0, y_max]))
    )
    
    # Bar labels with darker color for visibility
    bar_labels = bars.mark_text(
        dy=-8,
        fontSize=11,
        fontWeight='bold',
        color="#0A6BBA"  # Dark blue for better visibility
    ).encode(
        text=alt.Text('TTP_NET:Q', format='.1f')
    )
    
    # Rolling average line - keeping red as requested
    avg_line = alt.Chart(rollingAvg_clean).mark_line(
        color='#DC143C',  # Crimson red
        strokeWidth=3,
        strokeDash=[5, 5]
    ).encode(
        x=alt.X('Month:O'),
        y=alt.Y('TTP:Q', scale=alt.Scale(domain=[0, y_max]))
    )
    
    # Time limit line - keeping orange
    limit_line = alt.Chart(pd.DataFrame({
        'Month': list(range(1, 13)),
        'limit': [time_limit] * 12
    })).mark_line(
        color='#FF8C00',  # Dark orange
        strokeWidth=2
    ).encode(
        x=alt.X('Month:O'),
        y=alt.Y('limit:Q', scale=alt.Scale(domain=[0, y_max]))
    )
    
    # Triangle positioned next to the rolling average line
    triangle = alt.Chart(triangle_data).mark_text(
        dx=15,   # Small offset to the right of the data point
        dy=-5,   # Slightly above the rolling average line
        fontSize=25,  # Bigger arrow
        color='orange',
        fontWeight='bold'
    ).encode(
        x=alt.X('Month:O'),
        y=alt.Y('TTP:Q', scale=alt.Scale(domain=[0, y_max])),
        text='Triangle:N'
    )
    
    # Annotation text positioned above the triangle
    annotation = alt.Chart(triangle_data).mark_text(
        dx=15,   # Aligned with triangle
        dy=-25,  # Above the triangle
        fontSize=12,
        fontWeight='bold',
        color='#1B5390',
        align='left'
    ).encode(
        x=alt.X('Month:O'),
        y=alt.Y('TTP:Q', scale=alt.Scale(domain=[0, y_max])),
        text='Comment:N'
    )
    
    # Create unified legend data
    legend_data = pd.DataFrame({
        'Legend': ['Monthly Values', 'Rolling Average', 'Contractual Limit'],
        'Month': [1, 1, 1],  # Dummy values for positioning
        'Value': [0, 0, 0],   # Dummy values
        'Color': ['#4682B4', '#DC143C', '#FF8C00']
    })
    
    # Create legend as separate marks
    legend_bars = alt.Chart(legend_data).mark_rect(
        width=15,
        height=15
    ).encode(
        x=alt.X('Legend:N', title=None, axis=alt.Axis(labelAngle=0)),
        color=alt.Color('Color:N', scale=None, legend=None)
    ).properties(
        width=400,
        height=30
    ).resolve_scale(color='independent')
    
    # Main chart
    main_chart = (bars + bar_labels + avg_line + limit_line + triangle + annotation).properties(
        width=600,
        height=300,
        title=alt.TitleParams(
            text=f'{prog} {paymentType} - Time to Pay Analysis',
            fontSize=16,
            fontWeight='bold',
            anchor='start',
            color='#1B5390'
        )
    ).resolve_scale(
        color='independent'
    )
    
    # Combine main chart with legend
    final_chart = alt.vconcat(
        main_chart,
        legend_bars
    ).resolve_scale(
        color='independent'
    )
    
    return final_chart


# =============================================================================
# FINAL GENERATION FUNCTIONS
# =============================================================================

def generate_charts(df_paym, cutoff, quarterly_tables):
    """
    Generate TTP charts for each programme and payment type, skipping empty tables
    """
    charts = {}
    payment_types = ['IP', 'FP', 'EXPERTS', 'PF']  # Short form values
    programs = ['H2020', 'HEU']

    for prog in programs:
        for pt in payment_types:
            table_key = f'{prog}_{pt}_table'
            # Skip chart generation if the corresponding table is empty
            if quarterly_tables[table_key]['is_empty']:
                print(f"Skipping chart for {table_key} (empty table)")
                continue

            # Prepare data for chart
            df_chart = df_paym[
                (df_paym['Programme'] == prog) &
                (df_paym['v_payment_type'] == pt)
            ].copy()
            df_chart['Month'] = pd.to_datetime(df_chart['Pay Document Date (dd/mm/yyyy)']).dt.month
            df_chart['TTP_NET'] = pd.to_numeric(df_chart['v_TTP_NET'], errors='coerce')
            df_chart = df_chart[df_chart['TTP_NET'] >= 0]

            # If df_chart is empty, this should be caught by the table check above, but confirm
            if df_chart.empty:
                print(f"Data for chart {table_key} is empty, but table flag not set correctly")
                continue

            # Calculate average for this combination
            current_metrics = calculate_current_ttp_metrics(df_chart, cutoff)
            avg_ttp_net = round(float(current_metrics[prog][pt]['avg_ttp_net']),1)if pt in current_metrics[prog] else 0
            rolling_avg = rolling_ttp(df_chart, prog, pt)

            df_ttp = avg_ttp(df_chart, prog, pt)

            # Generate chart
            chart = chart_machine_ttp(df_ttp, pt, prog, avg_ttp_net, rolling_avg)
            logger.debug(f"Generated tta_chart_img for {prog, '{prog}_{pt}_chart'}")
            
            var_name = f'{prog}_{pt}_ttp_chart'
            try:
                logger.debug(f"Saving {var_name} to database")
                insert_variable(
                    report=report, module="PaymentsModule", var=var_name,
                    value=df_ttp,
                    db_path=db_path, anchor=var_name, altair_chart=chart
                )
                logger.debug(f"Saved {var_name} to database")
            except Exception as e:
                logger.error(f"Failed to save {var_name}: {str(e)}")

            charts[f'{prog}_{pt}_chart'] = chart
            

    return charts

# =============================================================================
# MAIN USAGE FOR CHARTS
# =============================================================================

def main_charts(df_paym, cutoff, quarterly_tables):
    """
    Main function to generate and display TTP charts, skipping empty tables
    """
    # Generate charts
    ttp_charts = generate_charts(df_paym, cutoff, quarterly_tables)
    
    # Display charts (in Jupyter Notebook)
    for key, chart in ttp_charts.items():
        print(f"\n{key}:")
        display(chart)
    
    return ttp_charts

# Example usage (uncomment to run after generating tables)
# ttp_charts = main_charts(df_paym, cutoff, quarterly_tables)


In [ ]:
ttp_charts = main_charts(df_paym, cutoff, quarterly_tables)

In [ ]:
def create_effectiveness_breakdown(df_paym, cutoff):
    """
    Create effectiveness breakdown tables by directorate and payment type
    """
    
    # Filter data up to cutoff and deduplicate by Pay Payment Key
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]

    df_filtered = df_paym[
        df_paym['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
    ].copy()
    df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
    
    # Convert to numeric
    df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
    df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
    df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')
    
    # Filter out negative TTP_NET values
    df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]
    
    # Determine year label from cutoff
    cutoff_date = pd.to_datetime(cutoff)
    current_year = cutoff_date.year
    year_label = f"{current_year} - YTD"
    
    return df_unique, year_label

def create_quarterly_ttp_breakdown(df_paym, cutoff, metric_type='NET'):
    """
    Create quarterly TTP breakdown tables by directorate and payment type
    
    Parameters:
    - df_paym: Payment data DataFrame
    - cutoff: Cutoff date for analysis
    - metric_type: 'NET' or 'GROSS' for which TTP metric to use
    """
    
    # Filter data up to cutoff and deduplicate by Pay Payment Key
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]

    df_filtered = df_paym[
        df_paym['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
    ].copy()
    df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
    
    # Convert to numeric
    df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
    df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
    df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')
    
    # Filter out negative TTP_NET values
    df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]
    
    # Extract quarter from date
    df_unique['Quarter'] = pd.to_datetime(df_unique['Pay Document Date (dd/mm/yyyy)']).dt.to_period('Q')
    
    # Create quarter labels (e.g., 2024Q1, 2024Q2, etc.)
    df_unique['Quarter_Label'] = df_unique['Quarter'].astype(str)
    
    # Get unique quarters and sort them
    quarters = sorted(df_unique['Quarter_Label'].unique())
    
    return quarters, df_unique

def create_h2020_quarterly_table(df_paym, cutoff, metric_type='NET'):
    """
    Create H2020 quarterly breakdown table with MultiIndex columns
    """
    quarters, df_unique = create_quarterly_ttp_breakdown(df_paym, cutoff, metric_type)
    
    # Filter for H2020
    h2020_data = df_unique[df_unique['Programme'] == 'H2020'].copy()
    
    # Get actual directorates from the data
    available_directorates = sorted([d for d in h2020_data['call_type'].unique() if pd.notna(d)])
    
    # Define payment types based on the image structure
    payment_types_h2020 = ['FP', 'IP']  # Final Payments and Interim Payments
    
    # Create MultiIndex columns structure
    columns = []
    
    # Add directorate columns
    for dir_name in available_directorates:
        for pt in payment_types_h2020:
            columns.append((dir_name, pt))
    
    # Add Experts column (single column, no sub-payment types)
    columns.append(('Experts', 'Experts'))
    
    # Add Total column  
    columns.append(('Total', 'Total'))
    
    multi_columns = pd.MultiIndex.from_tuples(columns, names=['Directorate', 'Payment_Type'])
    
    # Create DataFrame with quarters as index
    quarters_with_summary = quarters + ['Dep C.']
    quarterly_data = pd.DataFrame(index=quarters_with_summary, columns=multi_columns, dtype=float)
    
    ttp_column = f'v_TTP_{metric_type}'
    
    # Fill quarterly data
    for quarter in quarters:
        quarter_data = h2020_data[h2020_data['Quarter_Label'] == quarter]
        
        # Fill directorate data
        for dir_name in available_directorates:
            dir_data = quarter_data[quarter_data['call_type'] == dir_name]
            
            for pt in payment_types_h2020:
                pt_data = dir_data[dir_data['v_payment_type'] == pt]
                avg_ttp = pt_data[ttp_column].mean() if len(pt_data) > 0 else 0
                quarterly_data.loc[quarter, (dir_name, pt)] = round(avg_ttp, 1) if not pd.isna(avg_ttp) else 0.0
        
        # Experts
        experts_data = quarter_data[quarter_data['v_payment_type'] == 'EXPERTS']
        experts_avg = experts_data[ttp_column].mean() if len(experts_data) > 0 else 0
        quarterly_data.loc[quarter, ('Experts', 'Experts')] = round(experts_avg, 1) if not pd.isna(experts_avg) else 0.0
        
        # Total
        total_avg = quarter_data[ttp_column].mean() if len(quarter_data) > 0 else 0
        quarterly_data.loc[quarter, ('Total', 'Total')] = round(total_avg, 1) if not pd.isna(total_avg) else 0.0
    
    # Fill Dep C. summary row
    for dir_name in available_directorates:
        dir_data = h2020_data[h2020_data['call_type'] == dir_name]
        
        for pt in payment_types_h2020:
            pt_data = dir_data[dir_data['v_payment_type'] == pt]
            avg_ttp = pt_data[ttp_column].mean() if len(pt_data) > 0 else 0
            quarterly_data.loc['Dep C.', (dir_name, pt)] = round(avg_ttp, 1) if not pd.isna(avg_ttp) else 0.0
    
    # Overall experts and total for Dep C.
    experts_overall = h2020_data[h2020_data['v_payment_type'] == 'EXPERTS'][ttp_column].mean()
    quarterly_data.loc['Dep C.', ('Experts', 'Experts')] = round(experts_overall, 1) if not pd.isna(experts_overall) else 0.0
    
    total_overall = h2020_data[ttp_column].mean()
    quarterly_data.loc['Dep C.', ('Total', 'Total')] = round(total_overall, 1) if not pd.isna(total_overall) else 0.0
    
    # Convert to numeric and handle any remaining NaN values
    quarterly_data = quarterly_data.fillna(0.0).infer_objects(copy=False).astype(float)
    
    return quarterly_data

def create_heu_quarterly_table(df_paym, cutoff, metric_type='NET'):
    """
    Create HEU quarterly breakdown table with MultiIndex columns
    """
    quarters, df_unique = create_quarterly_ttp_breakdown(df_paym, cutoff, metric_type)
    
    # Filter for HEU
    heu_data = df_unique[df_unique['Programme'] == 'HEU'].copy()
    
    # Get actual directorates from the data
    available_directorates = sorted([d for d in heu_data['call_type'].unique() if pd.notna(d)])
    
    # Create MultiIndex columns structure based on image
    columns = []
    
    # Based on image: ADG(PF,IP), COG(PF,FP), POC(PF,IP), STG(PF,IP), SYG(PF)
    directorate_payment_mapping = {
        'ADG': ['PF', 'IP'],
        'COG': ['PF', 'FP'], 
        'POC': ['PF', 'IP'],
        'STG': ['PF', 'IP'],
        'SYG': ['PF']
    }
    
    # Add columns for available directorates
    for dir_name in available_directorates:
        if dir_name in directorate_payment_mapping:
            for pt in directorate_payment_mapping[dir_name]:
                columns.append((dir_name, pt))
    
    # Add Experts column
    columns.append(('Experts', 'Experts'))
    
    # Add Total column  
    columns.append(('Total', 'Total'))
    
    multi_columns = pd.MultiIndex.from_tuples(columns, names=['Directorate', 'Payment_Type'])
    
    # Create DataFrame with quarters as index
    quarters_with_summary = quarters + ['Dep C.']
    quarterly_data = pd.DataFrame(index=quarters_with_summary, columns=multi_columns, dtype=float)

    
    ttp_column = f'v_TTP_{metric_type}'
    
    # Fill quarterly data
    for quarter in quarters:
        quarter_data = heu_data[heu_data['Quarter_Label'] == quarter]
        
        # Fill directorate data
        for dir_name in available_directorates:
            if dir_name in directorate_payment_mapping:
                dir_data = quarter_data[quarter_data['call_type'] == dir_name]
                
                for pt in directorate_payment_mapping[dir_name]:
                    pt_data = dir_data[dir_data['v_payment_type'] == pt]
                    avg_ttp = pt_data[ttp_column].mean() if len(pt_data) > 0 else 0
                    quarterly_data.loc[quarter, (dir_name, pt)] = round(avg_ttp, 1) if not pd.isna(avg_ttp) else 0.0
        
        # Experts
        experts_data = quarter_data[quarter_data['v_payment_type'] == 'EXPERTS']
        experts_avg = experts_data[ttp_column].mean() if len(experts_data) > 0 else 0
        quarterly_data.loc[quarter, ('Experts', 'Experts')] = round(experts_avg, 1) if not pd.isna(experts_avg) else 0.0
        
        # Total
        total_avg = quarter_data[ttp_column].mean() if len(quarter_data) > 0 else 0
        quarterly_data.loc[quarter, ('Total', 'Total')] = round(total_avg, 1) if not pd.isna(total_avg) else 0.0
    
    # Fill Dep C. summary row
    for dir_name in available_directorates:
        if dir_name in directorate_payment_mapping:
            dir_data = heu_data[heu_data['call_type'] == dir_name]
            
            for pt in directorate_payment_mapping[dir_name]:
                pt_data = dir_data[dir_data['v_payment_type'] == pt]
                avg_ttp = pt_data[ttp_column].mean() if len(pt_data) > 0 else 0
                quarterly_data.loc['Dep C.', (dir_name, pt)] = round(avg_ttp, 1) if not pd.isna(avg_ttp) else 0.0
    
    # Overall experts and total for Dep C.
    experts_overall = heu_data[heu_data['v_payment_type'] == 'EXPERTS'][ttp_column].mean()
    quarterly_data.loc['Dep C.', ('Experts', 'Experts')] = round(experts_overall, 1) if not pd.isna(experts_overall) else 0.0
    
    total_overall = heu_data[ttp_column].mean()
    quarterly_data.loc['Dep C.', ('Total', 'Total')] = round(total_overall, 1) if not pd.isna(total_overall) else 0.0
    
    # Convert to numeric and handle any remaining NaN values
    quarterly_data = quarterly_data.fillna(0.0).infer_objects(copy=False).astype(float)
    
    return quarterly_data

def generate_quarterly_breakdown_tables(df_paym, cutoff, metric_type='NET'):
    """
    Generate both H2020 and HEU quarterly breakdown tables with MultiIndex columns
    
    Usage in Jupyter:
    h2020_table, heu_table = generate_quarterly_breakdown_tables(df_paym, cutoff, 'NET')
    
    # Display tables
    print("H2020 Quarterly Breakdown:")
    display(h2020_table)
    
    print("\nHEU Quarterly Breakdown:")
    display(heu_table)
    
    # Access MultiIndex structure:
    print("HEU columns:", heu_table.columns.tolist())
    print("Data for ADG-PF:", heu_table[('ADG', 'PF')])
    """
    h2020_table = create_h2020_quarterly_table(df_paym, cutoff, metric_type)
    heu_table = create_heu_quarterly_table(df_paym, cutoff, metric_type)
    
    return h2020_table, heu_table

def create_h2020_effectiveness_table(df_paym, cutoff):
    """
    Create H2020 effectiveness breakdown table by directorate
    """
    df_unique, year_label = create_effectiveness_breakdown(df_paym, cutoff)
    
    # Filter for H2020
    h2020_data = df_unique[df_unique['Programme'] == 'H2020'].copy()
    
    # Get actual directorates from the data
    available_directorates = sorted([d for d in h2020_data['call_type'].unique() if pd.notna(d)])
    
    # Build table data
    table_data = []
    
    # Calculate effectiveness for each directorate
    for dir_name in available_directorates:
        dir_data = h2020_data[h2020_data['call_type'] == dir_name]
        row = {'call_type': dir_name}
        
        # Final Payments (30 days target)
        fp_data = dir_data[dir_data['v_payment_type'] == 'FP']
        if len(fp_data) > 0:
            fp_valid = fp_data[fp_data['v_payment_in_time'].notna()]
            fp_effectiveness = (fp_data['v_payment_in_time'].sum() / len(fp_valid) * 100) if len(fp_valid) > 0 else 0
            row[f'Final Payments - The contractual time limit is 30 days {year_label}'] = f"{fp_effectiveness:.2f}%"
        else:
            row[f'Final Payments - The contractual time limit is 30 days {year_label}'] = "-"
        
        # Interim Payments (60 days target)
        ip_data = dir_data[dir_data['v_payment_type'] == 'IP']
        if len(ip_data) > 0:
            ip_valid = ip_data[ip_data['v_payment_in_time'].notna()]
            ip_effectiveness = (ip_data['v_payment_in_time'].sum() / len(ip_valid) * 100) if len(ip_valid) > 0 else 0
            row[f'Interim Payments - The contractual time limit is 60 days {year_label}'] = f"{ip_effectiveness:.2f}%"
        else:
            row[f'Interim Payments - The contractual time limit is 60 days {year_label}'] = "-"
        
        # Experts Payment (30 days target)
        exp_data = dir_data[dir_data['v_payment_type'] == 'EXPERTS']
        if len(exp_data) > 0:
            exp_valid = exp_data[exp_data['v_payment_in_time'].notna()]
            exp_effectiveness = (exp_data['v_payment_in_time'].sum() / len(exp_valid) * 100) if len(exp_valid) > 0 else 0
            row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = f"{exp_effectiveness:.2f}%"
        else:
            row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = "-"
        
        # All Payments (60 days target)
        if len(dir_data) > 0:
            all_valid = dir_data[dir_data['v_payment_in_time'].notna()]
            all_effectiveness = (dir_data['v_payment_in_time'].sum() / len(all_valid) * 100) if len(all_valid) > 0 else 0
            row[f'All Payments - The contractual time limit is 60 days {year_label}'] = f"{all_effectiveness:.2f}%"
        else:
            row[f'All Payments - The contractual time limit is 60 days {year_label}'] = "-"
        
        table_data.append(row)
    
    # Add "All Payments" summary row
    all_row = {'call_type': 'All Payments'}
    
    # Calculate overall effectiveness for all directorates
    # Final Payments
    all_fp_data = h2020_data[h2020_data['v_payment_type'] == 'FP']
    if len(all_fp_data) > 0:
        all_fp_valid = all_fp_data[all_fp_data['v_payment_in_time'].notna()]
        all_fp_effectiveness = (all_fp_data['v_payment_in_time'].sum() / len(all_fp_valid) * 100) if len(all_fp_valid) > 0 else 0
        all_row[f'Final Payments - The contractual time limit is 30 days {year_label}'] = f"{all_fp_effectiveness:.2f}%"
    else:
        all_row[f'Final Payments - The contractual time limit is 30 days {year_label}'] = "-"
    
    # Interim Payments
    all_ip_data = h2020_data[h2020_data['v_payment_type'] == 'IP']
    if len(all_ip_data) > 0:
        all_ip_valid = all_ip_data[all_ip_data['v_payment_in_time'].notna()]
        all_ip_effectiveness = (all_ip_data['v_payment_in_time'].sum() / len(all_ip_valid) * 100) if len(all_ip_valid) > 0 else 0
        all_row[f'Interim Payments - The contractual time limit is 60 days {year_label}'] = f"{all_ip_effectiveness:.2f}%"
    else:
        all_row[f'Interim Payments - The contractual time limit is 60 days {year_label}'] = "-"
    
    # Experts Payment
    all_exp_data = h2020_data[h2020_data['v_payment_type'] == 'EXPERTS']
    if len(all_exp_data) > 0:
        all_exp_valid = all_exp_data[all_exp_data['v_payment_in_time'].notna()]
        all_exp_effectiveness = (all_exp_data['v_payment_in_time'].sum() / len(all_exp_valid) * 100) if len(all_exp_valid) > 0 else 0
        all_row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = f"{all_exp_effectiveness:.2f}%"
    else:
        all_row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = "-"
    
    # All Payments
    if len(h2020_data) > 0:
        all_h2020_valid = h2020_data[h2020_data['v_payment_in_time'].notna()]
        all_h2020_effectiveness = (h2020_data['v_payment_in_time'].sum() / len(all_h2020_valid) * 100) if len(all_h2020_valid) > 0 else 0
        all_row[f'All Payments - The contractual time limit is 60 days {year_label}'] = f"{all_h2020_effectiveness:.2f}%"
    else:
        all_row[f'All Payments - The contractual time limit is 60 days {year_label}'] = "-"
    
    table_data.append(all_row)
    
    return pd.DataFrame(table_data)

def create_heu_effectiveness_table(df_paym, cutoff):
    """
    Create HEU effectiveness breakdown table by directorate
    """
    df_unique, year_label = create_effectiveness_breakdown(df_paym, cutoff)
    
    # Filter for HEU
    heu_data = df_unique[df_unique['Programme'] == 'HEU'].copy()
    
    # Get actual directorates from the data
    available_directorates = sorted([d for d in heu_data['call_type'].unique() if pd.notna(d)])
    
    # Build table data
    table_data = []
    
    # Calculate effectiveness for each directorate
    for dir_name in available_directorates:
        dir_data = heu_data[heu_data['call_type'] == dir_name]
        row = {'call_type': dir_name}
        
        # Experts Payment (30 days target)
        exp_data = dir_data[dir_data['v_payment_type'] == 'EXPERTS']
        if len(exp_data) > 0:
            exp_valid = exp_data[exp_data['v_payment_in_time'].notna()]
            exp_effectiveness = (exp_data['v_payment_in_time'].sum() / len(exp_valid) * 100) if len(exp_valid) > 0 else 0
            row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = f"{exp_effectiveness:.2f}%"
        else:
            row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = "-"
        
        # Final Payments (90 days target)
        fp_data = dir_data[dir_data['v_payment_type'] == 'FP']
        if len(fp_data) > 0:
            fp_valid = fp_data[fp_data['v_payment_in_time'].notna()]
            fp_effectiveness = (fp_data['v_payment_in_time'].sum() / len(fp_valid) * 100) if len(fp_valid) > 0 else 0
            row[f'Final Payments - The contractual time limit is 90 days {year_label}'] = f"{fp_effectiveness:.2f}%"
        else:
            row[f'Final Payments - The contractual time limit is 90 days {year_label}'] = "-"
        
        # Interim Payments (90 days target)
        ip_data = dir_data[dir_data['v_payment_type'] == 'IP']
        if len(ip_data) > 0:
            ip_valid = ip_data[ip_data['v_payment_in_time'].notna()]
            ip_effectiveness = (ip_data['v_payment_in_time'].sum() / len(ip_valid) * 100) if len(ip_valid) > 0 else 0
            row[f'Interim Payments - The contractual time limit is 90 days {year_label}'] = f"{ip_effectiveness:.2f}%"
        else:
            row[f'Interim Payments - The contractual time limit is 90 days {year_label}'] = "-"
        
        # Prefinancing Payments (days target - need to determine from data)
        pf_data = dir_data[dir_data['v_payment_type'] == 'PF']
        if len(pf_data) > 0:
            pf_valid = pf_data[pf_data['v_payment_in_time'].notna()]
            pf_effectiveness = (pf_data['v_payment_in_time'].sum() / len(pf_valid) * 100) if len(pf_valid) > 0 else 0
            row[f'Prefinancing Payments - The contractual time limit is days {year_label}'] = f"{pf_effectiveness:.2f}%"
        else:
            row[f'Prefinancing Payments - The contractual time limit is days {year_label}'] = "-"
        
        # All Payments (90 days target)
        if len(dir_data) > 0:
            all_valid = dir_data[dir_data['v_payment_in_time'].notna()]
            all_effectiveness = (dir_data['v_payment_in_time'].sum() / len(all_valid) * 100) if len(all_valid) > 0 else 0
            row[f'All Payments - The contractual time limit is 90 days {year_label}'] = f"{all_effectiveness:.2f}%"
        else:
            row[f'All Payments - The contractual time limit is 90 days {year_label}'] = "-"
        
        table_data.append(row)
    
    # Add "All Payments" summary row
    all_row = {'call_type': 'All Payments'}
    
    # Calculate overall effectiveness for all directorates
    # Experts Payment
    all_exp_data = heu_data[heu_data['v_payment_type'] == 'EXPERTS']
    if len(all_exp_data) > 0:
        all_exp_valid = all_exp_data[all_exp_data['v_payment_in_time'].notna()]
        all_exp_effectiveness = (all_exp_data['v_payment_in_time'].sum() / len(all_exp_valid) * 100) if len(all_exp_valid) > 0 else 0
        all_row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = f"{all_exp_effectiveness:.2f}%"
    else:
        all_row[f'Experts Payment - The contractual time limit is 30 days {year_label}'] = "-"
    
    # Final Payments
    all_fp_data = heu_data[heu_data['v_payment_type'] == 'FP']
    if len(all_fp_data) > 0:
        all_fp_valid = all_fp_data[all_fp_data['v_payment_in_time'].notna()]
        all_fp_effectiveness = (all_fp_data['v_payment_in_time'].sum() / len(all_fp_valid) * 100) if len(all_fp_valid) > 0 else 0
        all_row[f'Final Payments - The contractual time limit is 90 days {year_label}'] = f"{all_fp_effectiveness:.2f}%"
    else:
        all_row[f'Final Payments - The contractual time limit is 90 days {year_label}'] = "-"
    
    # Interim Payments
    all_ip_data = heu_data[heu_data['v_payment_type'] == 'IP']
    if len(all_ip_data) > 0:
        all_ip_valid = all_ip_data[all_ip_data['v_payment_in_time'].notna()]
        all_ip_effectiveness = (all_ip_data['v_payment_in_time'].sum() / len(all_ip_valid) * 100) if len(all_ip_valid) > 0 else 0
        all_row[f'Interim Payments - The contractual time limit is 90 days {year_label}'] = f"{all_ip_effectiveness:.2f}%"
    else:
        all_row[f'Interim Payments - The contractual time limit is 90 days {year_label}'] = "-"
    
    # Prefinancing Payments
    all_pf_data = heu_data[heu_data['v_payment_type'] == 'PF']
    if len(all_pf_data) > 0:
        all_pf_valid = all_pf_data[all_pf_data['v_payment_in_time'].notna()]
        all_pf_effectiveness = (all_pf_data['v_payment_in_time'].sum() / len(all_pf_valid) * 100) if len(all_pf_valid) > 0 else 0
        all_row[f'Prefinancing Payments - The contractual time limit is days {year_label}'] = f"{all_pf_effectiveness:.2f}%"
    else:
        all_row[f'Prefinancing Payments - The contractual time limit is days {year_label}'] = "-"
    
    # All Payments
    if len(heu_data) > 0:
        all_heu_valid = heu_data[heu_data['v_payment_in_time'].notna()]
        all_heu_effectiveness = (heu_data['v_payment_in_time'].sum() / len(all_heu_valid) * 100) if len(all_heu_valid) > 0 else 0
        all_row[f'All Payments - The contractual time limit is 90 days {year_label}'] = f"{all_heu_effectiveness:.2f}%"
    else:
        all_row[f'All Payments - The contractual time limit is 90 days {year_label}'] = "-"
    
    table_data.append(all_row)
    
    return pd.DataFrame(table_data)


def generate_effectiveness_breakdown_tables(df_paym, cutoff):
    """
    Generate both H2020 and HEU effectiveness breakdown tables (ORIGINAL STRUCTURE)
    
    Usage in Jupyter:
    h2020_eff_table, heu_eff_table = generate_effectiveness_breakdown_tables(df_paym, cutoff)
    
    # Display tables
    print("H2020 Effectiveness Breakdown:")
    display(h2020_eff_table)
    
    print("\nHEU Effectiveness Breakdown:")
    display(heu_eff_table)
    """
    h2020_eff_table = create_h2020_effectiveness_table(df_paym, cutoff)
    heu_eff_table = create_heu_effectiveness_table(df_paym, cutoff)
    
    return h2020_eff_table, heu_eff_table

# =============================================================================
# CLEAN TTP CALCULATION FUNCTIONS (from original file)
# =============================================================================

def calculate_current_ttp_metrics(df_paym, cutoff):
    """
    Calculate current TTP metrics from df_paym data
    """
    # Filter data up to cutoff and deduplicate by Pay Payment Key
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]

    df_filtered = df_paym[
        df_paym['Pay Document Date (dd/mm/yyyy)'] <= last_valid_date
    ].copy()
    df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
    
    # Convert to numeric
    df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
    df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
    df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')

    # Filter out negative TTP_NET values
    df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]
    
    results = {}
    
    # Calculate by Programme and Payment Type
    for programme in ['H2020', 'HEU']:
        prog_data = df_unique[df_unique['Programme'] == programme]
        if len(prog_data) == 0:
            continue
            
        results[programme] = {}
        
        # Overall programme metrics
        prog_valid = prog_data[prog_data['v_payment_in_time'].notna()]
        results[programme]['overall'] = {
            'avg_ttp_net': prog_data['v_TTP_NET'].mean(),
            'avg_ttp_gross': prog_data['v_TTP_GROSS'].mean(),
            'on_time_pct': prog_data['v_payment_in_time'].sum() / len(prog_valid) if len(prog_valid) > 0 else 0
        }
        
        # By payment type - using correct short form values from v_payment_type
        payment_types = ['IP', 'FP', 'EXPERTS', 'PF']  # Short form values
        
        for payment_type in payment_types:
            pt_data = prog_data[prog_data['v_payment_type'] == payment_type]
            if len(pt_data) > 0:
                pt_valid = pt_data[pt_data['v_payment_in_time'].notna()]
                results[programme][payment_type] = {
                    'avg_ttp_net': pt_data['v_TTP_NET'].mean(),
                    'avg_ttp_gross': pt_data['v_TTP_GROSS'].mean(),
                    'on_time_pct': pt_data['v_payment_in_time'].sum() / len(pt_valid) if len(pt_valid) > 0 else 0
                }
    
    # Overall total
    total_valid = df_unique[df_unique['v_payment_in_time'].notna()]
    results['TOTAL'] = {
        'avg_ttp_net': df_unique['v_TTP_NET'].mean(),
        'avg_ttp_gross': df_unique['v_TTP_GROSS'].mean(),
        'on_time_pct': df_unique['v_payment_in_time'].sum() / len(total_valid) if len(total_valid) > 0 else 0
    }
    
    return results

def load_historical_ttp_data(report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Load historical TTP data from database
    """
    DB_PATH = Path(db_path)
    report_params = load_report_params(report_name=report_name, db_path=DB_PATH)
    
    return {
        "TTP_NET_HISTORY": report_params.get("TTP_NET_HISTORY"),
        "TTP_GROSS_HISTORY": report_params.get("TTP_GROSS_HISTORY"),
        "PAYMENTS_ON_TIME_HISTORY": report_params.get("PAYMENTS_ON_TIME_HISTORY")
    }

def create_ttp_comparison_table(df_paym, cutoff, historical_data):
    """
    Create TTP comparison table matching the image structure
    """
    # Calculate current metrics
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    
    # Determine labels based on cutoff
    cutoff_date = pd.to_datetime(cutoff)
    current_year = cutoff_date.year
    current_label = f"{current_year}-YTD"
    historical_label = f"Dec {current_year - 1}"
    
    # Build comparison data
    comparison_data = []
    
    # H2020 section
    h2020_current = current_metrics.get('H2020', {})
    
    # H2020 - Interim Payments (IP)
    current_ip = h2020_current.get('IP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Interim Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_ip['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("IP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_ip['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("IP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_ip['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('IP', 0)*100:.2f}%"
    })
    
    # H2020 - Final Payments (FP)
    current_fp = h2020_current.get('FP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Final Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_fp['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("FP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_fp['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("FP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_fp['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('FP', 0)*100:.2f}%"
    })
    
    # H2020 - Experts Payments (EXPERTS)
    current_exp = h2020_current.get('EXPERTS', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Experts Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_exp['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("Experts", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_exp['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("Experts", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_exp['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('Experts', 0)*100:.2f}%"
    })
    
    # H2020 - Overall
    current_h2020 = h2020_current.get('overall', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'H2020',
        f'Average Net Time to Pay (in days) {current_label}': round(current_h2020['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("H2020", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_h2020['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("H2020", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_h2020['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('H2020', 0)*100:.2f}%"
    })
    
    # HEU section
    heu_current = current_metrics.get('HEU', {})
    
    # HEU - Prefinancing Payments (PF)
    current_pf = heu_current.get('PF', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Prefinancing Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_pf['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("PF", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_pf['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("PF", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_pf['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('PF', 0)*100:.2f}%"
    })
    
    # HEU - Interim Payments (IP)
    current_ip_heu = heu_current.get('IP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Interim Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_ip_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("IP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_ip_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("IP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_ip_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('IP', 0)*100:.2f}%"
    })
    
    # HEU - Final Payments (FP)
    current_fp_heu = heu_current.get('FP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Final Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_fp_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("FP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_fp_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("FP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_fp_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('FP', 0)*100:.2f}%"
    })
    
    # HEU - Experts Payment (EXPERTS)
    current_exp_heu = heu_current.get('EXPERTS', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Experts Payment',
        f'Average Net Time to Pay (in days) {current_label}': round(current_exp_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("Experts", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_exp_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("Experts", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_exp_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('Experts', 0)*100:.2f}%"
    })
    
    # HEU - Overall
    current_heu = heu_current.get('overall', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'HEU',
        f'Average Net Time to Pay (in days) {current_label}': round(current_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("HEU", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("HEU", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('HEU', 0)*100:.2f}%"
    })
    
    # TOTAL row
    current_total = current_metrics.get('TOTAL', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'TOTAL',
        f'Average Net Time to Pay (in days) {current_label}': round(current_total['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["ALL"].get("TOTAL", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_total['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["ALL"].get("TOTAL", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_total['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['ALL'].get('TOTAL', 0)*100:.2f}%"
    })
    
    return pd.DataFrame(comparison_data)

def create_ttp_effectiveness_table(df_paym, cutoff, historical_data, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Create TTP effectiveness and efficiency indicators table
    """
    # Calculate current metrics
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    
    # Determine labels based on cutoff using get_scope_start_end
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]
    current_month = pd.to_datetime(last_valid_date).strftime('%b-%y')
    
    cutoff_date = pd.to_datetime(cutoff)
    current_year = cutoff_date.year
    historical_label = f"Dec-{str(current_year-1)[-2:]}"
    
    # Load database parameters for admin and expert meetings
    from pathlib import Path
    DB_PATH = Path(db_path)
    report_params = load_report_params(report_name=report_name, db_path=DB_PATH)
    
    admin_eff = report_params.get("Administrative_expenditure_Effectiveness", {})
    admin_ttp = report_params.get("Administrative_expenditure_ttp", {})
    expt_meet_eff = report_params.get("Expert_meetings_Effectiveness", {})
    expt_meet_ttp = report_params.get("Expert_meetings_ttp", {})
    
    # Build effectiveness data
    effectiveness_data = []
    
    # Get current metrics for calculations
    h2020_current = current_metrics.get('H2020', {})
    heu_current = current_metrics.get('HEU', {})
    
    # Research grants - Interim Payments - H2020
    h2020_ip = h2020_current.get('IP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Interim Payments - H2020',
        current_month: f"{h2020_ip*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('IP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Research grants - Final Payments - H2020
    h2020_fp = h2020_current.get('FP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Final Payments - H2020',
        current_month: f"{h2020_fp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('FP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Experts with Appointment Letters H2020
    h2020_exp = h2020_current.get('EXPERTS', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Experts with Appointment Letters H2020',
        current_month: f"{h2020_exp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('Experts', 0)*100:.2f}%",
        'Target': '95% in 30 days'
    })
    
    # Research grants - Pre-financings HEU
    heu_pf = heu_current.get('PF', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Pre-financings HEU',
        current_month: f"{heu_pf*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('PF', 0)*100:.2f}%",
        'Target': '95% in 30 days'
    })
    
    # Research grants - Interim Payments HEU
    heu_ip = heu_current.get('IP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Interim Payments HEU',
        current_month: f"{heu_ip*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('IP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Research grants - Final Payments HEU
    heu_fp = heu_current.get('FP', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Research grants - Final Payments HEU',
        current_month: f"{heu_fp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('FP', 0)*100:.2f}%",
        'Target': '95% in 90 days'
    })
    
    # Administrative expenditure (from database)
    admin_current = admin_eff.get("Current", 0)
    admin_old = admin_eff.get("Old", 0)
    admin_target = admin_eff.get("Target", "99% in 30 days")
    
    # Format admin values - handle both percentage (0.985) and already formatted (98.5%) values
    if isinstance(admin_current, (int, float)) and admin_current <= 1:
        admin_current_str = f"{admin_current*100:.2f}%"
    else:
        admin_current_str = str(admin_current) if admin_current != "na" else "n/a"
        
    if isinstance(admin_old, (int, float)) and admin_old <= 1:
        admin_old_str = f"{admin_old*100:.2f}%"
    else:
        admin_old_str = str(admin_old) if admin_old != "na" else "n/a"
    
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Administrative expenditure',
        current_month: admin_current_str,
        historical_label: admin_old_str,
        'Target': admin_target
    })
    
    # Experts with Appointment Letters HEU
    heu_exp = heu_current.get('EXPERTS', {}).get('on_time_pct', 0)
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Experts with Appointment Letters HEU',
        current_month: f"{heu_exp*100:.2f}%",
        historical_label: f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('Experts', 0)*100:.2f}%",
        'Target': '95% in 30 days'
    })
    
    # Expert meetings PMO (from database)
    expt_current = expt_meet_eff.get("Current", "na")
    expt_old = expt_meet_eff.get("Old", "na")
    expt_target = expt_meet_eff.get("Target", "n/a")
    
    # Format expert values - handle both percentage and string values
    if isinstance(expt_current, (int, float)) and expt_current <= 1:
        expt_current_str = f"{expt_current*100:.2f}%"
    else:
        expt_current_str = str(expt_current) if expt_current != "na" else "n/a"
        
    if isinstance(expt_old, (int, float)) and expt_old <= 1:
        expt_old_str = f"{expt_old*100:.2f}%"
    else:
        expt_old_str = str(expt_old) if expt_old != "na" else "n/a"
    
    effectiveness_data.append({
        'Time-to–Pay: % of payments made on time (H2020 - HEU)': 'Expert meetings PMO',
        current_month: expt_current_str,
        historical_label: expt_old_str,
        'Target': expt_target
    })
    
    return pd.DataFrame(effectiveness_data)

def create_ttp_days_table(df_paym, cutoff, historical_data, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Create Time to Pay: Average number of days (H2020 - HEU) table
    """
    # Calculate current metrics
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    
    # Load database parameters for admin and expert meetings
    from pathlib import Path
    DB_PATH = Path(db_path)
    report_params = load_report_params(report_name=report_name, db_path=DB_PATH)
    
    admin_eff = report_params.get("Administrative_expenditure_Effectiveness", {})
    admin_ttp = report_params.get("Administrative_expenditure_ttp", {})
    expt_meet_eff = report_params.get("Expert_meetings_Effectiveness", {})
    expt_meet_ttp = report_params.get("Expert_meetings_ttp", {})
    
    # Build days data
    days_data = []
    
    # Get current metrics for calculations
    h2020_current = current_metrics.get('H2020', {})
    heu_current = current_metrics.get('HEU', {})
    
    # Research grants (days) - Interim Payments- H2020
    h2020_ip = h2020_current.get('IP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Interim Payments- H2020',
        'NET': round(h2020_ip.get('avg_ttp_net', 0), 1),
        'GROSS': round(h2020_ip.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Research grants (days) - Final Payments- H2020
    h2020_fp = h2020_current.get('FP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Final Payments- H2020',
        'NET': round(h2020_fp.get('avg_ttp_net', 0), 1),
        'GROSS': round(h2020_fp.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Experts with Appointment Letters (days) H2020
    h2020_exp = h2020_current.get('EXPERTS', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Experts with Appointment Letters (days) H2020',
        'NET': round(h2020_exp.get('avg_ttp_net', 0), 1),
        'GROSS': round(h2020_exp.get('avg_ttp_gross', 0), 1),
        'Target': '30 days'
    })
    
    # Research grants (days) - Pre-financings - HEU
    heu_pf = heu_current.get('PF', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Pre-financings - HEU',
        'NET': round(heu_pf.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_pf.get('avg_ttp_gross', 0), 1),
        'Target': '30 days'
    })
    
    # Research grants (days) - Interim Payments- HEU
    heu_ip = heu_current.get('IP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Interim Payments- HEU',
        'NET': round(heu_ip.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_ip.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Research grants (days) - Final Payments- HEU
    heu_fp = heu_current.get('FP', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Research grants (days) - Final Payments- HEU',
        'NET': round(heu_fp.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_fp.get('avg_ttp_gross', 0), 1),
        'Target': '90 days'
    })
    
    # Expert with Appointment Letter (days) HEU
    heu_exp = heu_current.get('EXPERTS', {})
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Expert with Appointment Letter (days) HEU',
        'NET': round(heu_exp.get('avg_ttp_net', 0), 1),
        'GROSS': round(heu_exp.get('avg_ttp_gross', 0), 1),
        'Target': '30 days'
    })
    
    # Expert meetings (PMO) (days) - from database
    expt_net_current = expt_meet_ttp.get("Current", "na")
    expt_gross_current = expt_meet_ttp.get("Current", "na")  # Assuming same for both NET and GROSS
    
    # Format expert meeting values
    if isinstance(expt_net_current, (int, float)):
        expt_net_str = str(round(expt_net_current, 1))
        expt_gross_str = str(round(expt_gross_current, 1))
    else:
        expt_net_str = "n/a"
        expt_gross_str = "n/a"
    
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Expert meetings (PMO) (days)',
        'NET': expt_net_str,
        'GROSS': expt_gross_str,
        'Target': '30 days'
    })
    
    # Administrative expenditure (days) - from database
    admin_net_current = admin_ttp.get("Current", 0)
    admin_gross_current = admin_ttp.get("Current", 0)  # Assuming same for both NET and GROSS
    
    # Format admin values
    if isinstance(admin_net_current, (int, float)):
        admin_net_str = str(round(admin_net_current, 1))
        admin_gross_str = str(round(admin_gross_current, 1))
    else:
        admin_net_str = "n/a"
        admin_gross_str = "n/a"
    
    days_data.append({
        'Time to Pay: Average number of days (H2020 - HEU)': 'Administrative expenditure (days)',
        'NET': admin_net_str,
        'GROSS': admin_gross_str,
        'Target': '30 days'
    })
    
    return pd.DataFrame(days_data)

def generate_all_ttp_tables(df_paym, cutoff, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Generate all three TTP tables - comparison table, effectiveness table, and days table
    
    Usage in Jupyter:
    comparison_table, effectiveness_table, days_table = generate_all_ttp_tables(df_paym, cutoff)
    """
    # Load historical data
    historical_data = load_historical_ttp_data(report_name=report_name, db_path=db_path)
    
    # Create all three tables
    comparison_table = create_ttp_comparison_table(df_paym, cutoff, historical_data)
    effectiveness_table = create_ttp_effectiveness_table(df_paym, cutoff, historical_data, report_name, db_path)
    days_table = create_ttp_days_table(df_paym, cutoff, historical_data, report_name, db_path)
    
    return comparison_table, effectiveness_table, days_table

def generate_complete_ttp_suite(df_paym, cutoff, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Generate the complete TTP report suite with all table variations
    
    Usage in Jupyter:
    tables = generate_complete_ttp_suite(df_paym, cutoff)
    
    # Access any table:
    comparison_table = tables['comparison']
    effectiveness_table = tables['effectiveness'] 
    days_table = tables['days']
    h2020_net_quarterly = tables['h2020_net_quarterly']  # MultiIndex DataFrame
    heu_net_quarterly = tables['heu_net_quarterly']      # MultiIndex DataFrame  
    h2020_gross_quarterly = tables['h2020_gross_quarterly']
    heu_gross_quarterly = tables['heu_gross_quarterly']
    h2020_effectiveness = tables['h2020_effectiveness']  # Clean table like Picture 1
    heu_effectiveness = tables['heu_effectiveness']      # Clean table like Picture 1
    
    # Example: Access specific data from MultiIndex quarterly table
    print("ADG PF data:", tables['heu_net_quarterly'][('ADG', 'PF')])
    """
    
    # Generate summary tables
    comparison_table, effectiveness_table, days_table = generate_all_ttp_tables(df_paym, cutoff, report_name, db_path)
    
    # Generate quarterly breakdown tables (days) with MultiIndex - FOR GREAT_TABLES
    h2020_net_quarterly, heu_net_quarterly = generate_quarterly_breakdown_tables(df_paym, cutoff, 'NET')
    h2020_gross_quarterly, heu_gross_quarterly = generate_quarterly_breakdown_tables(df_paym, cutoff, 'GROSS')
    
    # Generate effectiveness breakdown tables (percentages) - CLEAN STRUCTURE like Picture 1
    h2020_effectiveness, heu_effectiveness = generate_effectiveness_breakdown_tables(df_paym, cutoff)
    
    return {
        'comparison': comparison_table,
        'effectiveness': effectiveness_table,
        'days': days_table,
        'h2020_net_quarterly': h2020_net_quarterly,
        'heu_net_quarterly': heu_net_quarterly,
        'h2020_gross_quarterly': h2020_gross_quarterly,
        'heu_gross_quarterly': heu_gross_quarterly,
        'h2020_effectiveness': h2020_effectiveness,
        'heu_effectiveness': heu_effectiveness
    }

# =============================================================================
# UPDATED MAIN FUNCTIONS - USE THESE IN JUPYTER
# =============================================================================

def generate_ttp_tables(df_paym, cutoff, report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Main function to generate all TTP tables
    
    Usage in Jupyter:
    comparison_table, effectiveness_table, days_table = generate_ttp_tables(df_paym, cutoff)
    
    # Display all three summary tables
    print("TTP Comparison Table:")
    display(comparison_table)
    
    print("\nEffectiveness and Efficiency Indicators:")
    display(effectiveness_table)
    
    print("\nTime to Pay: Average number of days:")
    display(days_table)
    """
    return generate_all_ttp_tables(df_paym, cutoff, report_name, db_path)

def generate_quarterly_tables_for_great_tables(df_paym, cutoff):
    """
    Generate quarterly breakdown tables specifically formatted for great_tables (MultiIndex)
    
    Usage in Jupyter:
    h2020_net, heu_net, h2020_gross, heu_gross = generate_quarterly_tables_for_great_tables(df_paym, cutoff)
    
    # These DataFrames have MultiIndex columns perfect for great_tables:
    # - Level 0: Directorate (ADG, COG, POC, STG, SYG, Experts, Total)  
    # - Level 1: Payment Type (PF, IP, FP, EXPERTS)
    # - Index: Quarters (2024Q1, 2024Q2, etc.) + "Dep C." summary row
    
    # Example usage:
    print("HEU NET quarterly table structure:")
    print("Columns:", heu_net.columns.tolist())
    print("Index:", heu_net.index.tolist())
    print("ADG-PF column:", heu_net[('ADG', 'PF')])
    """
    h2020_net, heu_net = generate_quarterly_breakdown_tables(df_paym, cutoff, 'NET')
    h2020_gross, heu_gross = generate_quarterly_breakdown_tables(df_paym, cutoff, 'GROSS')
    
    return h2020_net, heu_net, h2020_gross, heu_gross

def generate_effectiveness_tables_for_display(df_paym, cutoff):
    """
    Generate effectiveness breakdown tables for regular display (Clean structure like Picture 1)
    
    Usage in Jupyter:
    h2020_eff, heu_eff = generate_effectiveness_tables_for_display(df_paym, cutoff)
    
    # These DataFrames have clean structure like Picture 1:
    # - Rows: Directorates (ADG, COG, POC, STG, SYG, All Payments)
    # - Columns: Payment types with descriptive headers
    # - Values: Formatted percentages like "100.00%"
    
    # Example usage:
    print("H2020 Effectiveness Table:")
    display(h2020_eff)
    """
    return generate_effectiveness_breakdown_tables(df_paym, cutoff)



In [ ]:
h2020_net, heu_net, h2020_gross, heu_gross = generate_quarterly_tables_for_great_tables(df_paym, cutoff)


In [ ]:
heu_net

In [ ]:
tables = generate_complete_ttp_suite(df_paym, cutoff)
h2020_effect = tables['h2020_effectiveness']
h2020_effect 

In [ ]:
heu_effect = tables['heu_effectiveness']
heu_effect

In [104]:
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime, date
from typing import Tuple
import locale
import warnings
warnings.filterwarnings('ignore')

# Set locale for number formatting (adjust as needed)
try:
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
except:
    try:
        locale.setlocale(locale.LC_ALL, 'C')
    except:
        pass

# Configuration - Update these as needed
colorScheme = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
title_color = '#333333'

def get_current_reporting_date():
    """Get current reporting date - uses current timestamp"""
    return pd.Timestamp.now()

def get_call_type_column(df):
    """Identify the call type column in the dataframe"""
    possible_columns = ['SCRS_CALL_TYPE', 'call_type', 'Call Type', 'v_call_type', 'Local Position', 'v_1_Program']
    for col in possible_columns:
        if col in df.columns:
            return col
    return None

def prepare_payment_data(df_paym, programme, call_type=None, cutoff_date=None):
    """
    Prepare payment data for chart generation using existing date utilities
    
    Parameters:
    - df_paym: Payment dataframe
    - programme: 'H2020' or 'HEU' 
    - call_type: Specific call type (e.g., 'STG', 'ADG', etc.) or 'all' for all
    - cutoff_date: Cutoff date for reporting (default: current timestamp)
    """
    
    if cutoff_date is None:
        cutoff_date = get_current_reporting_date()
    
    # Ensure cutoff_date is a pandas Timestamp
    if isinstance(cutoff_date, str):
        cutoff_date = pd.to_datetime(cutoff_date)
    elif hasattr(cutoff_date, 'date'):  # datetime object
        cutoff_date = pd.Timestamp(cutoff_date)
    elif not isinstance(cutoff_date, pd.Timestamp):
        cutoff_date = pd.Timestamp(cutoff_date)
    
    print(f"Preparing data for {programme} - {call_type} - Cutoff: {cutoff_date}")
    
    # Use existing utilities to get scope
    reporting_year = determine_epoch_year(cutoff_date)
    scope_start, scope_end = get_scope_start_end(cutoff_date)
    months_list = months_in_scope(cutoff_date)
    
    print(f"Reporting year: {reporting_year}")
    print(f"Scope: {scope_start} to {scope_end}")
    print(f"Months in scope: {len(months_list)} months")
    
    # Copy and filter data
    df = df_paym.copy()
    
    # Filter by programme
    df = df[df['Programme'] == programme].copy()
    
    # Parse dates if they're strings
    if df['Pay Document Date (dd/mm/yyyy)'].dtype == 'object':
        df['Pay Document Date (dd/mm/yyyy)'] = pd.to_datetime(df['Pay Document Date (dd/mm/yyyy)'], 
                                                              format='%d/%m/%Y', errors='coerce')
    
    # Filter by date scope (use scope_start and scope_end)
    df = df[
        (df['Pay Document Date (dd/mm/yyyy)'] >= scope_start) & 
        (df['Pay Document Date (dd/mm/yyyy)'] <= scope_end)
    ].copy()
    
    # Filter by fund source (equivalent to old C1, E0 filter)
    df = df[df['Fund Source'].notna()].copy()
    
    # Handle call type filtering
    call_type_col = get_call_type_column(df)
    
    if call_type and call_type != 'all' and call_type_col:
        df = df[df[call_type_col] == call_type].copy()
        print(f"Filtered by {call_type_col} = {call_type}: {len(df)} rows")
    
    if len(df) == 0:
        print("No data after filtering")
        return create_dummy_payment_data(programme, call_type, cutoff_date, reporting_year)
    
    # Create month column
    df['Month'] = df['Pay Document Date (dd/mm/yyyy)'].dt.month
    df['Year'] = df['Pay Document Date (dd/mm/yyyy)'].dt.year
    
    # Filter for reporting year (already filtered by scope, but double-check)
    df = df[df['Year'] == reporting_year].copy()
    
    # Aggregate by month (sum amounts)
    monthly_payments = df.groupby(['Month'])['v_amount_to_sum'].sum().reset_index()
    monthly_payments.rename(columns={'v_amount_to_sum': 'Paid'}, inplace=True)
    
    # Add programme and year info
    monthly_payments['v_1_Program'] = programme
    monthly_payments['Year'] = reporting_year
    
    print(f"Monthly payments aggregated: {len(monthly_payments)} months")
    return monthly_payments

def prepare_forecast_data(df_forecast, programme, call_type=None):
    """
    Prepare forecast data to match payment data structure
    """
    
    df = df_forecast.copy()
    
    # Map the actual column names from your forecast data
    programme_col_map = {
        'SCRS_FMWK': 'SCRS_FMWK',  # Your actual programme column
        'Programme': 'Programme',
        'v_1_Program': 'v_1_Program'
    }
    
    call_type_col_map = {
        'SCRS_CALL_TYPE': 'SCRS_CALL_TYPE',  # Your actual call type column
        'call_type': 'call_type',
        'Local Position': 'Local Position'
    }
    
    forecast_amount_col_map = {
        'Sum(SCRS_C1_MNT)': 'Sum(SCRS_C1_MNT)',  # Your actual forecast amount column
        'Forecast': 'Forecast',
        'v_amount_to_sum': 'v_amount_to_sum'
    }
    
    month_col_map = {
        'Month_Num': 'Month_Num',  # Your actual month number column
        'Month': 'Month'
    }
    
    # Find programme column
    programme_col = None
    for col in programme_col_map.keys():
        if col in df.columns:
            programme_col = col
            break
    
    # Find call type column  
    call_type_col = None
    for col in call_type_col_map.keys():
        if col in df.columns:
            call_type_col = col
            break
    
    # Find forecast amount column
    forecast_col = None
    for col in forecast_amount_col_map.keys():
        if col in df.columns:
            forecast_col = col
            break
            
    # Find month column
    month_col = None
    for col in month_col_map.keys():
        if col in df.columns:
            month_col = col
            break
    
    print(f"Forecast data columns detected:")
    print(f"  Programme: {programme_col}")
    print(f"  Call Type: {call_type_col}")  
    print(f"  Amount: {forecast_col}")
    print(f"  Month: {month_col}")
    
    # Filter by programme
    if programme_col:
        df = df[df[programme_col] == programme].copy()
        print(f"Filtered forecast by {programme_col} = {programme}: {len(df)} rows")
    else:
        print("No programme column found in forecast data - using all data")
    
    # Handle call type filtering
    if call_type and call_type != 'all' and call_type_col:
        df = df[df[call_type_col] == call_type].copy()
        print(f"Filtered forecast by {call_type_col} = {call_type}: {len(df)} rows")
    
    if len(df) == 0:
        print("No forecast data after filtering - creating zero forecast")
        return pd.DataFrame({
            'Month': range(1, 13),
            'Forecast': [0] * 12,
            'v_1_Program': programme
        })
    
    if not forecast_col:
        print(f"No forecast amount column found. Available columns: {list(df.columns)}")
        print("Creating zero forecast")
        return pd.DataFrame({
            'Month': range(1, 13),
            'Forecast': [0] * 12,
            'v_1_Program': programme
        })
    
    # Aggregate by month if month column exists
    if month_col:
        forecast_monthly = df.groupby([month_col])[forecast_col].sum().reset_index()
        forecast_monthly.rename(columns={
            month_col: 'Month',
            forecast_col: 'Forecast'
        }, inplace=True)
    else:
        # Distribute forecast equally across 12 months
        total_forecast = df[forecast_col].sum()
        forecast_monthly = pd.DataFrame({
            'Month': range(1, 13),
            'Forecast': [total_forecast / 12] * 12
        })
    
    forecast_monthly['v_1_Program'] = programme
    
    print(f"Forecast monthly data prepared: {len(forecast_monthly)} months, total: {forecast_monthly['Forecast'].sum():,.0f}")
    return forecast_monthly

def create_dummy_payment_data(programme, call_type, cutoff_date, reporting_year):
    """Create dummy data when no payments exist"""
    
    # Use the existing months_in_scope function to get the right months
    months_list = months_in_scope(cutoff_date)
    month_numbers = list(range(1, len(months_list) + 1))
    
    dummy_data = []
    
    for month in month_numbers:
        dummy_data.append({
            'Month': month,
            'Paid': 0,
            'v_1_Program': programme
        })
    
    return pd.DataFrame(dummy_data)

def merge_payment_and_forecast_data(df_paid, df_forecast, programme, budget_amount=None):
    """
    Merge payment and forecast data, calculate cumulative values and deviations
    """
    
    # Merge on Month
    df_merged = pd.merge(df_paid, df_forecast[['Month', 'Forecast']], 
                        on='Month', how='outer').fillna(0)
    
    # Ensure we have all months 1-12
    all_months = pd.DataFrame({'Month': range(1, 13)})
    df_merged = pd.merge(all_months, df_merged, on='Month', how='left').fillna(0)
    df_merged['v_1_Program'] = programme
    
    # Sort by month
    df_merged.sort_values('Month', inplace=True)
    df_merged.reset_index(drop=True, inplace=True)
    
    # Calculate cumulative values
    df_merged['Paid_Cumulative'] = df_merged['Paid'].cumsum()
    df_merged['Forecast_Cumulative'] = df_merged['Forecast'].cumsum()
    
    # Add budget appropriations only if available
    df_merged['Has_Budget'] = budget_amount is not None and budget_amount > 0
    if df_merged['Has_Budget'].iloc[0]:
        df_merged['Appropriations'] = budget_amount
    else:
        df_merged['Appropriations'] = None
    
    # Calculate total forecast for percentage calculations
    total_forecast = df_merged['Forecast'].sum()
    
    if total_forecast > 0:
        df_merged['Consumption_Pct'] = df_merged['Paid_Cumulative'] / total_forecast
        df_merged['Forecast_Pct'] = df_merged['Forecast_Cumulative'] / total_forecast
        df_merged['Deviation_Pct'] = df_merged['Consumption_Pct'] - df_merged['Forecast_Pct']
    else:
        df_merged['Consumption_Pct'] = 0
        df_merged['Forecast_Pct'] = 0
        df_merged['Deviation_Pct'] = 0
    
    # Calculate deviation in absolute terms
    df_merged['Deviation_Amount'] = df_merged['Paid_Cumulative'] - df_merged['Forecast_Cumulative']
    
    # Calculate deviation vs budget percentage only if budget is available
    if df_merged['Has_Budget'].iloc[0]:
        df_merged['Deviation_vs_Budget_Pct'] = df_merged['Deviation_Amount'] / budget_amount
    else:
        df_merged['Deviation_vs_Budget_Pct'] = None
    
    return df_merged

def create_payment_chart(df_merged, programme, call_type, year, cutoff_month):
    """
    Create the payment consumption chart using Altair
    """
    
    # Filter data up to cutoff month for actual payments
    df_display = df_merged.copy()
    df_current = df_display[df_display['Month'] <= cutoff_month].copy()
    
    # Check if budget data is available
    has_budget = df_merged['Has_Budget'].iloc[0] if len(df_merged) > 0 else False
    
    # Chart title
    if call_type == 'all':
        title = f"{programme} - Consumption All Calls {year}"
    else:
        title = f"{programme} - Consumption {call_type} {year}"
    
    # Get current month deviation for annotation
    if len(df_current) > 0:
        current_deviation = df_current.iloc[-1]['Deviation_Pct']
        current_month = df_current.iloc[-1]['Month']
    else:
        current_deviation = 0
        current_month = cutoff_month
    
    # Determine deviation color and annotation
    if current_deviation < -0.01:  # Less than -1%
        deviation_color = 'red'
        deviation_comment = 'Underconsumption'
        arrow = '↓'
    elif current_deviation > 0.01:  # Greater than 1%
        deviation_color = 'green'
        deviation_comment = 'Overconsumption'
        arrow = '↑'
    else:
        deviation_color = '#2b7a30'
        deviation_comment = 'On Track'
        arrow = '→'
    
    # Set up Altair theme
    def my_theme():
        return {
            'config': {
                'view': {'continuousHeight': 350, 'continuousWidth': 550},
                'range': {'category': {'scheme': colorScheme}},
                'title': {
                    "fontSize": 18, 
                    "font": 'Lato', 
                    "anchor": "center",
                    'color': title_color,
                    'fontWeight': 'bold'
                }
            }
        }
    
    alt.themes.register('my_theme', my_theme)
    alt.themes.enable('my_theme')
    
    # Base chart for consumption bars
    bar_chart = alt.Chart(df_current, title=title).mark_bar(
        size=45,
        opacity=0.7,
        color='#1f77b4'
    ).encode(
        x=alt.X('Month:O', title='Month'),
        y=alt.Y('Paid_Cumulative:Q', title='Amount (€)', scale=alt.Scale(zero=True)),
    ).properties(
        width=550,
        height=350
    )
    
    # Forecast line
    forecast_line = alt.Chart(df_display).mark_line(
        opacity=0.8,
        strokeWidth=3,
        color='#ff7f0e'
    ).encode(
        x='Month:O',
        y='Forecast_Cumulative:Q'
    )
    
    # Appropriations line - only if budget data is available
    charts_to_combine = [bar_chart, forecast_line]
    
    if has_budget:
        appropriations_line = alt.Chart(df_display).mark_line(
            opacity=0.8,
            strokeWidth=3,
            color='#2ca02c'
        ).encode(
            x='Month:O',
            y='Appropriations:Q'
        )
        charts_to_combine.append(appropriations_line)
    
    # Percentage text on bars
    percentage_text = bar_chart.mark_text(
        baseline='bottom',
        dx=0,
        dy=-5,
        align='center',
        fontSize=12,
        fontWeight='bold',
        color='#8a2003'
    ).encode(
        text=alt.Text('Consumption_Pct:Q', format='.1%')
    )
    
    # Deviation annotation
    deviation_base = alt.Chart(pd.DataFrame([{
        'Month': current_month,
        'Paid_Cumulative': df_current.iloc[-1]['Paid_Cumulative'] if len(df_current) > 0 else 0,
        'Deviation_Pct': current_deviation,
        'Comment': deviation_comment,
        'Arrow': arrow
    }]))
    
    deviation_text = deviation_base.mark_text(
        dx=50,
        dy=-80,
        fontSize=14,
        fontWeight='bold',
        color=deviation_color,
        align='left'
    ).encode(
        x='Month:O',
        y='Paid_Cumulative:Q',
        text=alt.Text('Deviation_Pct:Q', format='.1%')
    )
    
    deviation_comment_text = deviation_base.mark_text(
        dx=50,
        dy=-60,
        fontSize=11,
        fontWeight='bold',
        color=deviation_color,
        align='left'
    ).encode(
        x='Month:O',
        y='Paid_Cumulative:Q',
        text='Comment:N'
    )
    
    # Add percentage text and deviation annotations
    charts_to_combine.extend([percentage_text, deviation_text, deviation_comment_text])
    
    # Combine all chart elements
    final_chart = alt.layer(*charts_to_combine)
    
    return final_chart

def create_summary_table(df_merged, cutoff_month):
    """
    Create the summary table below the chart - excludes budget rows when no budget data available
    """
    
    # Filter to current data
    df_table = df_merged[df_merged['Month'] <= cutoff_month].copy()
    
    # Check if budget data is available
    has_budget = df_merged['Has_Budget'].iloc[0] if len(df_merged) > 0 else False
    
    # Convert to millions for display
    df_table = df_table.copy()
    df_table['Paid_M'] = df_table['Paid_Cumulative'] / 1_000_000
    df_table['Forecast_M'] = df_table['Forecast_Cumulative'] / 1_000_000
    df_table['Deviation_M'] = df_table['Deviation_Amount'] / 1_000_000
    
    # Format numbers
    df_table['Paid_Formatted'] = df_table['Paid_M'].map('{:,.3f}'.format)
    df_table['Forecast_Formatted'] = df_table['Forecast_M'].map('{:,.3f}'.format)
    df_table['Consumption_Pct_Formatted'] = df_table['Consumption_Pct'].map('{:.1%}'.format)
    df_table['Forecast_Pct_Formatted'] = df_table['Forecast_Pct'].map('{:.1%}'.format)
    df_table['Deviation_Pct_Formatted'] = df_table['Deviation_Pct'].map('{:.1%}'.format)
    df_table['Deviation_M_Formatted'] = df_table['Deviation_M'].map('{:,.1f}'.format)
    
    # Define base columns for output table
    output_cols = [
        'Month', 'Paid_Formatted', 'Forecast_Formatted',
        'Deviation_M_Formatted', 'Deviation_Pct_Formatted'
    ]
    
    # Add budget-related columns only if budget data is available
    if has_budget:
        df_table['Budget_M'] = df_table['Appropriations'] / 1_000_000
        df_table['Budget_Formatted'] = df_table['Budget_M'].map('{:,.1f}'.format)
        df_table['Deviation_vs_Budget_Formatted'] = df_table['Deviation_vs_Budget_Pct'].map('{:.1%}'.format)
        
        # Insert budget column at the beginning and deviation vs budget at the end
        output_cols.insert(1, 'Budget_Formatted')
        output_cols.append('Deviation_vs_Budget_Formatted')
    
    # Create output table
    table_output = df_table[output_cols].copy()
    table_output.set_index('Month', inplace=True)
    
    # Transpose for month columns
    table_transposed = table_output.T
    table_transposed.reset_index(inplace=True)
    table_transposed.rename(columns={'index': 'Metric'}, inplace=True)
    
    # Define metric names based on whether budget data is available
    if has_budget:
        metric_names = {
            'Budget_Formatted': 'Current Inscribed Budget',
            'Paid_Formatted': 'Cumulative Consumption (€ million)',
            'Forecast_Formatted': 'Cumulative Forecast (€ million)',
            'Deviation_M_Formatted': 'Deviation vs. Cumulative Forecast (€ million)',
            'Deviation_Pct_Formatted': 'Deviation vs. Cumulative Forecast (%)',
            'Deviation_vs_Budget_Formatted': 'Deviation vs. Budget (%)'
        }
    else:
        metric_names = {
            'Paid_Formatted': 'Cumulative Consumption (€ million)',
            'Forecast_Formatted': 'Cumulative Forecast (€ million)',
            'Deviation_M_Formatted': 'Deviation vs. Cumulative Forecast (€ million)',
            'Deviation_Pct_Formatted': 'Deviation vs. Cumulative Forecast (%)'
        }
    
    table_transposed['Metric'] = table_transposed['Metric'].map(metric_names)
    
    # Rename month columns
    month_names = {
        1: 'JAN', 2: 'FEB', 3: 'MAR', 4: 'APR', 5: 'MAY', 6: 'JUN',
        7: 'JUL', 8: 'AUG', 9: 'SEP', 10: 'OCT', 11: 'NOV', 12: 'DEC'
    }
    
    table_transposed.columns = ['Metric'] + [month_names.get(col, str(col)) for col in table_transposed.columns[1:]]
    
    return table_transposed

def generate_payment_chart_and_table(df_paym, df_forecast, programme, call_type='all', 
                                   budget_amount=None, cutoff_date=None):
    """
    Main function to generate both chart and table for a specific programme/call type
    
    Parameters:
    - df_paym: Payment dataframe
    - df_forecast: Forecast dataframe  
    - programme: 'H2020' or 'HEU'
    - call_type: Specific call type or 'all'
    - budget_amount: Budget appropriation amount
    - cutoff_date: Reporting cutoff date
    
    Returns:
    - Dictionary with 'chart' and 'table' keys
    """
    
    try:
        # Get cutoff date
        if cutoff_date is None:
            cutoff_date = get_current_reporting_date()
        
        # Ensure cutoff_date is a pandas Timestamp
        if isinstance(cutoff_date, str):
            cutoff_date = pd.to_datetime(cutoff_date)
        elif not isinstance(cutoff_date, pd.Timestamp):
            cutoff_date = pd.Timestamp(cutoff_date)
        
        # Use existing date utilities
        reporting_year = determine_epoch_year(cutoff_date)
        scope_start, scope_end = get_scope_start_end(cutoff_date)
        months_list = months_in_scope(cutoff_date)
        cutoff_month = len(months_list)  # Number of months in scope
        
        print(f"\n=== Generating chart for {programme} - {call_type} ===")
        print(f"Cutoff date: {cutoff_date}")
        print(f"Reporting year: {reporting_year}")
        print(f"Scope: {scope_start} to {scope_end}")
        print(f"Months in scope: {cutoff_month}")
        
        # Prepare payment data
        df_paid = prepare_payment_data(df_paym, programme, call_type, cutoff_date)
        
        # Prepare forecast data
        df_forecast_prep = prepare_forecast_data(df_forecast, programme, call_type)
        
        # Merge and calculate
        df_merged = merge_payment_and_forecast_data(df_paid, df_forecast_prep, programme, budget_amount)
        
        # Create chart
        chart = create_payment_chart(df_merged, programme, call_type, reporting_year, cutoff_month)
        
        # Create table
        table = create_summary_table(df_merged, cutoff_month)
        
        return {
            'chart': chart,
            'table': table,
            'data': df_merged,
            'success': True,
            'metadata': {
                'reporting_year': reporting_year,
                'scope_start': scope_start,
                'scope_end': scope_end,
                'cutoff_month': cutoff_month,
                'months_in_scope': months_list
            }
        }
        
    except Exception as e:
        print(f"Error generating chart for {programme} - {call_type}: {str(e)}")
        import traceback
        traceback.print_exc()
        return {
            'chart': None,
            'table': None,
            'data': None,
            'success': False,
            'error': str(e)
        }

#### 
vars_all = fetch_vars_for_report(report, db_path)
vars_data = vars_all.get('table_2a_HE')
h2020_vars_data = vars_all.get('table_2a_H2020')

# Extract Total Available Payment Appropriations
heu_total_appropriations = next(
    (item['Available_Payment_Appropriations'] for item in vars_data 
     if item['Budget Address Type'] == 'Total'), 
    None
)
heu_total_appropriations_expt = next(
    (item['Available_Payment_Appropriations'] for item in vars_data 
     if item['Budget Address Type'] == 'Experts'), 
    None
)

h2020_total_appropriations = next(
    (item['Available_Payment_Appropriations'] for item in h2020_vars_data 
     if item['Budget Address Type'] == 'Total'), 
    None
)


# Budget configuration - Only available for specific cases
AVAILABLE_BUDGET_CONFIG = {
    'H2020': {
        'all': h2020_total_appropriations  # Total H2020 budget - ONLY available for 'all' call types
        # No individual call type budgets available (STG, ADG, COG, POC, SYG)
    },
    'HEU': {
        'all': heu_total_appropriations,    # Total HEU budget - available for 'all' call types
        'EXPERTS': heu_total_appropriations_expt   # HEU Experts budget - ONLY individual call type with budget
        # No other individual call type budgets available (STG, ADG, COG, POC, SYG)
    }
}

def check_budget_availability(programme, call_type):
    """
    Check if budget appropriations are available for the given programme/call_type combination
    
    Returns:
    - True if budget is available
    - False if no budget appropriations exist for this combination
    """
    
    if programme == 'H2020':
        return call_type == 'all'
    elif programme == 'HEU':
        return call_type in ['all', 'EXPERTS']
    else:
        return False

def get_budget_amount(programme, call_type, budget_config=None):
    """
    Get budget amount for programme/call_type if available
    
    Returns:
    - Budget amount if available
    - None if no budget appropriations exist
    """
    
    if budget_config is None:
        budget_config = AVAILABLE_BUDGET_CONFIG
    
    if not check_budget_availability(programme, call_type):
        return None
    
    try:
        return budget_config[programme][call_type]
    except KeyError:
        return None

def generate_all_charts(df_paym, df_forecast, programme_budgets=None, cutoff_date=None):
    """
    Generate charts for all available call types in both programmes
    Only applies budget appropriations where available (H2020 'all', HEU 'all' and 'EXPERTS')
    
    Parameters:
    - df_paym: Payment dataframe
    - df_forecast: Forecast dataframe
    - programme_budgets: Dict with budget amounts for available combinations only
    - cutoff_date: Reporting cutoff date
    """
    
    if programme_budgets is None:
        programme_budgets = AVAILABLE_BUDGET_CONFIG
    
    results = {}
    
    # Get available call types
    call_type_col = get_call_type_column(df_paym)
    
    if call_type_col:
        # Get unique call types per programme
        programmes = ['H2020', 'HEU']
        
        for programme in programmes:
            prog_data = df_paym[df_paym['Programme'] == programme]
            call_types = prog_data[call_type_col].dropna().unique().tolist()
            call_types.append('all')  # Add overall summary
            
            results[programme] = {}
            
            for call_type in call_types:
                # Get budget only if available for this combination
                budget = get_budget_amount(programme, call_type, programme_budgets)
                
                # Log budget availability
                if budget:
                    print(f"Budget available for {programme} {call_type}: €{budget:,.0f}")
                else:
                    print(f"No budget appropriations for {programme} {call_type} - skipping budget elements")
                
                result = generate_payment_chart_and_table(
                    df_paym, df_forecast, programme, call_type, budget, cutoff_date
                )
                
                results[programme][call_type] = result
                
                if result['success']:
                    budget_status = "with budget" if budget else "without budget"
                    print(f"✓ Generated: {programme} - {call_type} ({budget_status})")
                else:
                    print(f"✗ Failed: {programme} - {call_type}")
    
    return results

def save_charts_to_files(results, output_dir='./'):
    """
    Save generated charts to PNG files
    """
    
    for programme, prog_results in results.items():
        for call_type, result in prog_results.items():
            if result['success'] and result['chart'] is not None:
                filename = f"chart_{programme}_{call_type}.png"
                filepath = f"{output_dir}{filename}"
                
                try:
                    result['chart'].save(filepath)
                    print(f"Saved: {filepath}")
                except Exception as e:
                    print(f"Error saving {filepath}: {str(e)}")

def display_results_summary(results):
    """
    Display a summary of generated results
    """
    
    print("\n" + "="*60)
    print("CHART GENERATION SUMMARY")
    print("="*60)
    
    for programme, prog_results in results.items():
        print(f"\n{programme} Programme:")
        
        for call_type, result in prog_results.items():
            if result['success']:
                # Get key metrics from the data
                data = result['data']
                if data is not None and len(data) > 0:
                    total_paid = data['Paid_Cumulative'].max()
                    total_forecast = data['Forecast_Cumulative'].max()
                    final_deviation = data['Deviation_Pct'].iloc[-1] if len(data) > 0 else 0
                    has_budget = data['Has_Budget'].iloc[0] if len(data) > 0 else False
                    budget_status = "with budget" if has_budget else "no budget"
                    
                    print(f"  ✓ {call_type}: €{total_paid:,.0f} paid vs €{total_forecast:,.0f} forecast ({final_deviation:.1%} deviation) [{budget_status}]")
                else:
                    print(f"  ✓ {call_type}: Generated successfully")
            else:
                print(f"  ✗ {call_type}: {result.get('error', 'Unknown error')}")

# Main execution example
if __name__ == "__main__":
    print("Payment Charts and Tables Generator")
    print("="*50)
    print("This script requires df_paym and df_forecast to be loaded")
    print("\nIMPORTANT: Budget appropriations are only available for:")
    print("  H2020: 'all' call types combined")
    print("  HEU: 'all' call types combined and 'EXPERTS'")
    print("  Individual call types (STG, ADG, COG, POC, SYG) have NO budget data")
    print("\nExample usage:")
    print("# Generate all charts (budget applied where available)")
    print("results = generate_all_charts(df_paym, df_forecast, AVAILABLE_BUDGET_CONFIG)")
    print("display_results_summary(results)")
    print("save_charts_to_files(results)")
    print("\nOr generate individual charts:")
    print("# Individual call type (no budget elements)")
    print("result = generate_payment_chart_and_table(df_paym, df_forecast, 'HEU', 'STG', None)")
    print("# Programme total (with budget elements)")
    print("result = generate_payment_chart_and_table(df_paym, df_forecast, 'HEU', 'all', 1_532_500_000)")
    print("result['chart'].show()  # Display chart")
    print("print(result['table'])  # Display table")
    print("\nCheck budget availability:")
    print("print(check_budget_availability('HEU', 'STG'))     # False - no budget")
    print("print(check_budget_availability('HEU', 'all'))     # True - budget available")
    print("print(check_budget_availability('HEU', 'EXPERTS')) # True - budget available")

Payment Charts and Tables Generator
This script requires df_paym and df_forecast to be loaded

IMPORTANT: Budget appropriations are only available for:
  H2020: 'all' call types combined
  HEU: 'all' call types combined and 'EXPERTS'
  Individual call types (STG, ADG, COG, POC, SYG) have NO budget data

Example usage:
# Generate all charts (budget applied where available)
results = generate_all_charts(df_paym, df_forecast, AVAILABLE_BUDGET_CONFIG)
display_results_summary(results)
save_charts_to_files(results)

Or generate individual charts:
# Individual call type (no budget elements)
result = generate_payment_chart_and_table(df_paym, df_forecast, 'HEU', 'STG', None)
# Programme total (with budget elements)
result = generate_payment_chart_and_table(df_paym, df_forecast, 'HEU', 'all', 1_532_500_000)
result['chart'].show()  # Display chart
print(result['table'])  # Display table

Check budget availability:
print(check_budget_availability('HEU', 'STG'))     # False - no budget
print(check_

In [105]:
# Test the fixed chart generation
print("Testing fixed chart generation...")

# Test individual chart
result = generate_payment_chart_and_table(df_paym, df_forecast, 'HEU', 'STG')

if result['success']:
    print("✅ SUCCESS! Chart generated successfully")
    print(f"Metadata: {result['metadata']}")
    
    # Display the chart
    result['chart']
    
else:
    print(f"❌ Error: {result['error']}")

Testing fixed chart generation...

=== Generating chart for HEU - STG ===
Cutoff date: 2025-06-04 18:38:12.458901
Reporting year: 2025
Scope: 2025-01-01 00:00:00 to 2025-06-30 00:00:00
Months in scope: 5
Preparing data for HEU - STG - Cutoff: 2025-06-04 18:38:12.458901
Reporting year: 2025
Scope: 2025-01-01 00:00:00 to 2025-06-30 00:00:00
Months in scope: 5 months
Filtered by call_type = STG: 417 rows
Monthly payments aggregated: 6 months
Forecast data columns detected:
  Programme: SCRS_FMWK
  Call Type: SCRS_CALL_TYPE
  Amount: Sum(SCRS_C1_MNT)
  Month: Month_Num
Filtered forecast by SCRS_FMWK = HEU: 72 rows
Filtered forecast by SCRS_CALL_TYPE = STG: 12 rows
Forecast monthly data prepared: 12 months, total: 630,836,891
✅ SUCCESS! Chart generated successfully
Metadata: {'reporting_year': 2025, 'scope_start': Timestamp('2025-01-01 00:00:00'), 'scope_end': Timestamp('2025-06-30 00:00:00'), 'cutoff_month': 5, 'months_in_scope': ['January', 'February', 'March', 'April', 'May']}


In [106]:
# Test batch generation if individual tests work
if result['success']:
    print("\n" + "="*50)  
    print("Testing batch generation...")
    
    try:
        results = generate_all_charts(df_paym, df_forecast)
        print("✅ Batch generation successful!")
        display_results_summary(results)
    except Exception as e:
        print(f"❌ Batch generation failed: {str(e)}")


Testing batch generation...
No budget appropriations for H2020 COG - skipping budget elements

=== Generating chart for H2020 - COG ===
Cutoff date: 2025-06-04 18:38:17.812674
Reporting year: 2025
Scope: 2025-01-01 00:00:00 to 2025-06-30 00:00:00
Months in scope: 5
Preparing data for H2020 - COG - Cutoff: 2025-06-04 18:38:17.812674
Reporting year: 2025
Scope: 2025-01-01 00:00:00 to 2025-06-30 00:00:00
Months in scope: 5 months
Filtered by call_type = COG: 334 rows
Monthly payments aggregated: 6 months
Forecast data columns detected:
  Programme: SCRS_FMWK
  Call Type: SCRS_CALL_TYPE
  Amount: Sum(SCRS_C1_MNT)
  Month: Month_Num
Filtered forecast by SCRS_FMWK = H2020: 51 rows
Filtered forecast by SCRS_CALL_TYPE = COG: 12 rows
Forecast monthly data prepared: 12 months, total: 142,923,264
✓ Generated: H2020 - COG (without budget)
No budget appropriations for H2020 STG - skipping budget elements

=== Generating chart for H2020 - STG ===
Cutoff date: 2025-06-04 18:38:17.869750
Reporting ye

In [102]:
# 1. DISPLAY INDIVIDUAL CHARTS
print("=== DISPLAYING CHARTS ===")

# HEU STG Chart (no budget)
print("HEU STG Chart:")
results['HEU']['STG']['chart']

=== DISPLAYING CHARTS ===
HEU STG Chart:


alt.LayerChart(...)

In [88]:
# 2. Show its table  
display(results['HEU']['STG']['table'])

,Metric,JAN,FEB,MAR,APR,MAY
0,Cumulative Consumption (€ million),40.625,101.291,155.397,179.795,216.253
1,Cumulative Forecast (€ million),56.526,110.693,194.981,222.745,266.236
2,Deviation vs. Cumulative Forecast (€ million),-15.9,-9.4,-39.6,-43.0,-50.0
3,Deviation vs. Cumulative Forecast (%),-2.5%,-1.5%,-6.3%,-6.8%,-7.9%


In [107]:
# 3. Show HEU All chart (with budget line)
results['HEU']['all']['chart']

alt.LayerChart(...)

In [108]:


# 4. Show HEU All table (with budget rows)
display(results['HEU']['all']['table'])

,Metric,JAN,FEB,MAR,APR,MAY
0,Current Inscribed Budget,"2,066.5","2,066.5","2,066.5","2,066.5","2,066.5"
1,Cumulative Consumption (€ million),102.518,245.090,427.143,568.251,733.692
2,Cumulative Forecast (€ million),109.950,235.511,423.863,561.443,705.908
3,Deviation vs. Cumulative Forecast (€ million),-7.4,9.6,3.3,6.8,27.8
4,Deviation vs. Cumulative Forecast (%),-0.3%,0.4%,0.2%,0.3%,1.3%
5,Deviation vs. Budget (%),-0.4%,0.5%,0.2%,0.3%,1.3%
